# Description

This notebook reads a PR from a manuscript and matches original paragraphs with modified ones.

# Modules

In [1]:
from pathlib import Path

import pandas as pd
from github import Auth, Github
from IPython.display import display
from proj import conf
from proj.utils import process_paragraph

# Settings/paths

In [2]:
REPO = "pivlab/manubot-ai-editor-code-test-phenoplier-manuscript"
PR = (4, "gpt-3.5-turbo")

OUTPUT_FILE_PATH = None
REVERSED_OUTPUT_FILE_PATH = None

In [3]:
# Parameters
OUTPUT_FILE_PATH = "/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/phenoplier-manuscript--gpt-3.5-turbo.pkl"
REVERSED_OUTPUT_FILE_PATH = "/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/phenoplier-manuscript--gpt-3.5-turbo--reversed.pkl"

In [4]:
OUTPUT_FILE_PATH = Path(OUTPUT_FILE_PATH).resolve()
OUTPUT_FILE_PATH.parent.mkdir(parents=True, exist_ok=True)
display(OUTPUT_FILE_PATH)

PosixPath('/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/phenoplier-manuscript--gpt-3.5-turbo.pkl')

In [5]:
REVERSED_OUTPUT_FILE_PATH = Path(REVERSED_OUTPUT_FILE_PATH).resolve()
REVERSED_OUTPUT_FILE_PATH.parent.mkdir(parents=True, exist_ok=True)
display(REVERSED_OUTPUT_FILE_PATH)

PosixPath('/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/phenoplier-manuscript--gpt-3.5-turbo--reversed.pkl')

# Get Repo

In [6]:
auth = Auth.Token(conf.github.API_TOKEN)

In [7]:
g = Github(auth=auth)

In [8]:
repo = g.get_repo(REPO)

# Get Pull Request

In [9]:
pr = repo.get_pull(PR[0])

In [10]:
list(pr.get_files())

[File(sha="7b45c7b7750feb01fb8d7c476a7f000c333810e5", filename="content/01.abstract.md"),
 File(sha="1061b06fb8319082f2d5fe4dce4571bd27a0505e", filename="content/02.introduction.md"),
 File(sha="37c72c190d992a0802af4597ee5d3dc13f7d99c8", filename="content/04.05.00.results_framework.md"),
 File(sha="b0721dfbbc664a353baba59e44f9b26bfaa11204", filename="content/04.05.01.results_crispr.md"),
 File(sha="a74d6a0125a366a4c0fb7089f67cd2ee81c86c3d", filename="content/04.15.results_drug_disease_prediction.md"),
 File(sha="9b063b538a665c5efa1dcb6952930bd77f2e979e", filename="content/04.20.00.results_traits_clustering.md"),
 File(sha="ac236caa6554577b64cd88943e927f4fdff87d4e", filename="content/05.discussion.md"),
 File(sha="674d7fa2712c6b536cb6b36b9a0d90375e9c8687", filename="content/07.00.methods.md"),
 File(sha="fd6c4f43548c26c1f9b074d6585453e27d4dc171", filename="content/50.00.supplementary_material.md")]

In [11]:
pr_commits = list(pr.get_commits())

In [12]:
pr_commits[0].parents

[Commit(sha="11964b6420a11bf4e2d71325447b6af65c696471")]

In [13]:
pr_prev = pr_commits[0].parents[0].sha
print(pr_prev)

11964b6420a11bf4e2d71325447b6af65c696471


In [14]:
pr_curr = pr_commits[0].sha
print(pr_curr)

d0d51c89f01eeccdb26de4674beecb6005c33ae7


# Get file list

In [15]:
pr_files = [f for f in pr.get_files() if f.filename.endswith(".md")]
display(pr_files)

[File(sha="7b45c7b7750feb01fb8d7c476a7f000c333810e5", filename="content/01.abstract.md"),
 File(sha="1061b06fb8319082f2d5fe4dce4571bd27a0505e", filename="content/02.introduction.md"),
 File(sha="37c72c190d992a0802af4597ee5d3dc13f7d99c8", filename="content/04.05.00.results_framework.md"),
 File(sha="b0721dfbbc664a353baba59e44f9b26bfaa11204", filename="content/04.05.01.results_crispr.md"),
 File(sha="a74d6a0125a366a4c0fb7089f67cd2ee81c86c3d", filename="content/04.15.results_drug_disease_prediction.md"),
 File(sha="9b063b538a665c5efa1dcb6952930bd77f2e979e", filename="content/04.20.00.results_traits_clustering.md"),
 File(sha="ac236caa6554577b64cd88943e927f4fdff87d4e", filename="content/05.discussion.md"),
 File(sha="674d7fa2712c6b536cb6b36b9a0d90375e9c8687", filename="content/07.00.methods.md"),
 File(sha="fd6c4f43548c26c1f9b074d6585453e27d4dc171", filename="content/50.00.supplementary_material.md")]

# Sections

In [16]:
paragraph_matches = []

## Abstract

In [17]:
section_name = "abstract"

In [18]:
pr_filename = pr_files[0].filename
assert section_name in pr_filename
print(pr_filename)

content/01.abstract.md


### Original

In [19]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Abstract {.page_break_before}

Genes act in con


In [20]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

2

### Modified

In [21]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Abstract {.page_break_before}

How do genes int


In [22]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

2

### Match

In [23]:
orig_section_paragraphs[0]

'## Abstract {.page_break_before}'

In [24]:
mod_section_paragraphs[0]

'## Abstract {.page_break_before}'

####  Paragraph 00

In [25]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

Genes act in concert with each other in specific contexts to perform their functions. Determining how these genes influence complex traits requires a mechanistic understanding of expression regulation across different conditions. It has been shown that this insight is critical for developing new therapies. Transcriptome-wide association studies have helped uncover the role of individual genes in disease-relevant mechanisms. However, modern models of the architecture of complex traits predict that gene-gene interactions play a crucial role in disease origin and progression. Here we introduce PhenoPLIER, a computational approach that maps gene-trait associations and pharmacological perturbation data into a common latent representation for a joint analysis. This representation is based on modules of genes with similar expression patterns across the same conditions. We observe that diseases are significantly associated with gene modules expressed in relevant cell types, and our approach is

In [26]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

How do genes interact to influence complex traits and disease mechanisms, and how can this knowledge be leveraged for therapeutic development? This paper introduces PhenoPLIER, a computational approach that integrates gene-trait associations and pharmacological data to analyze gene expression patterns across different conditions. By identifying modules of co-expressed genes associated with diseases and drug mechanisms, PhenoPLIER can accurately predict drug-disease pairs and infer mechanisms of action. Through a CRISPR screen focused on lipid regulation, we demonstrate that PhenoPLIER can prioritize functionally important genes within trait-associated modules. This approach highlights the importance of considering gene-gene interactions in understanding disease etiology and identifying potential therapeutic targets for drug repurposing.


In [27]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [28]:
display(paragraph_matches[-1])

('abstract',
 'Genes act in concert with each other in specific contexts to perform their functions. Determining how these genes influence complex traits requires a mechanistic understanding of expression regulation across different conditions. It has been shown that this insight is critical for developing new therapies. Transcriptome-wide association studies have helped uncover the role of individual genes in disease-relevant mechanisms. However, modern models of the architecture of complex traits predict that gene-gene interactions play a crucial role in disease origin and progression. Here we introduce PhenoPLIER, a computational approach that maps gene-trait associations and pharmacological perturbation data into a common latent representation for a joint analysis. This representation is based on modules of genes with similar expression patterns across the same conditions. We observe that diseases are significantly associated with gene modules expressed in relevant cell types, and 

## Introduction

In [29]:
section_name = "introduction"

In [30]:
pr_filename = pr_files[1].filename
assert section_name in pr_filename
print(pr_filename)

content/02.introduction.md


### Original

In [31]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Introduction

Genes work together in context-sp


In [32]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

5

### Modified

In [33]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Introduction

Genes work together in specific n


In [34]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

5

### Match

In [35]:
orig_section_paragraphs[0]

'## Introduction'

In [36]:
mod_section_paragraphs[0]

'## Introduction'

####  Paragraph 00

In [37]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

Genes work together in context-specific networks to carry out different functions [@pmid:19104045; @doi:10.1038/ng.3259]. Variations in these genes can change their functional role and, at a higher level, affect disease-relevant biological processes [@doi:10.1038/s41467-018-06022-6]. In this context, determining how genes influence complex traits requires mechanistically understanding expression regulation across different cell types [@doi:10.1126/science.aaz1776; @doi:10.1038/s41586-020-2559-3; @doi:10.1038/s41576-019-0200-9], which in turn should lead to improved treatments [@doi:10.1038/ng.3314; @doi:10.1371/journal.pgen.1008489]. Previous studies have described different regulatory DNA elements [@doi:10.1038/nature11247; @doi:10.1038/nature14248; @doi:10.1038/nature12787; @doi:10.1038/s41586-020-03145-z; @doi:10.1038/s41586-020-2559-3] including genetic effects on gene expression across different tissues [@doi:10.1126/science.aaz1776]. Integrating functional genomics data and GWAS 

In [38]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

Genes work together in specific networks to carry out various functions. Changes in these genes can alter their function and, on a broader scale, impact biological processes relevant to disease. Understanding how genes influence complex traits requires a mechanistic understanding of gene expression regulation across different cell types, which can lead to improved treatments. Previous research has identified various regulatory DNA elements, including genetic effects on gene expression in different tissues. By integrating functional genomics data with GWAS data, researchers have improved the identification of transcriptional mechanisms that, when disrupted, often lead to tissue- and cell lineage-specific pathology.


In [39]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [40]:
display(paragraph_matches[-1])

('introduction',
 'Genes work together in context-specific networks to carry out different functions [@pmid:19104045; @doi:10.1038/ng.3259]. Variations in these genes can change their functional role and, at a higher level, affect disease-relevant biological processes [@doi:10.1038/s41467-018-06022-6]. In this context, determining how genes influence complex traits requires mechanistically understanding expression regulation across different cell types [@doi:10.1126/science.aaz1776; @doi:10.1038/s41586-020-2559-3; @doi:10.1038/s41576-019-0200-9], which in turn should lead to improved treatments [@doi:10.1038/ng.3314; @doi:10.1371/journal.pgen.1008489]. Previous studies have described different regulatory DNA elements [@doi:10.1038/nature11247; @doi:10.1038/nature14248; @doi:10.1038/nature12787; @doi:10.1038/s41586-020-03145-z; @doi:10.1038/s41586-020-2559-3] including genetic effects on gene expression across different tissues [@doi:10.1126/science.aaz1776]. Integrating functional geno

####  Paragraph 01

In [41]:
par0 = process_paragraph(orig_section_paragraphs[2])
print(par0)

Given the availability of gene expression data across several tissues [@doi:10.1038/nbt.3838; @doi:10.1038/s41467-018-03751-6; @doi:10.1126/science.aaz1776; @doi:10.1186/s13040-020-00216-9], an effective approach to identify these biological processes is the transcription-wide association study (TWAS), which integrates expression quantitative trait loci (eQTLs) data to provide a mechanistic interpretation for GWAS findings. TWAS relies on testing whether perturbations in gene regulatory mechanisms mediate the association between genetic variants and human diseases [@doi:10.1371/journal.pgen.1009482; @doi:10.1038/ng.3506; @doi:10.1371/journal.pgen.1007889; @doi:10.1038/ng.3367], and these approaches have been highly successful not only in understanding disease etiology at the transcriptome level [@pmid:33931583; @doi:10.1101/2021.10.21.21265225; @pmid:31036433] but also in disease-risk prediction (polygenic scores) [@doi:10.1186/s13059-021-02591-w] and drug repurposing [@doi:10.1038/nn.

In [42]:
par1 = process_paragraph(mod_section_paragraphs[2])
print(par1)

With the abundance of gene expression data available across various tissues, a useful method for identifying biological processes is the transcription-wide association study (TWAS). TWAS combines expression quantitative trait loci (eQTLs) data to offer a mechanistic explanation for GWAS findings. This approach examines whether disruptions in gene regulatory mechanisms mediate the connection between genetic variations and human diseases. These methods have proven successful in not only elucidating disease origins at the transcriptome level but also in predicting disease risk (polygenic scores) and repurposing drugs. Nevertheless, TWAS operates at the individual gene level and may not capture more intricate interactions at the network level.


In [43]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [44]:
display(paragraph_matches[-1])

('introduction',
 'Given the availability of gene expression data across several tissues [@doi:10.1038/nbt.3838; @doi:10.1038/s41467-018-03751-6; @doi:10.1126/science.aaz1776; @doi:10.1186/s13040-020-00216-9], an effective approach to identify these biological processes is the transcription-wide association study (TWAS), which integrates expression quantitative trait loci (eQTLs) data to provide a mechanistic interpretation for GWAS findings. TWAS relies on testing whether perturbations in gene regulatory mechanisms mediate the association between genetic variants and human diseases [@doi:10.1371/journal.pgen.1009482; @doi:10.1038/ng.3506; @doi:10.1371/journal.pgen.1007889; @doi:10.1038/ng.3367], and these approaches have been highly successful not only in understanding disease etiology at the transcriptome level [@pmid:33931583; @doi:10.1101/2021.10.21.21265225; @pmid:31036433] but also in disease-risk prediction (polygenic scores) [@doi:10.1186/s13059-021-02591-w] and drug repurposin

####  Paragraph 02

In [45]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

These gene-gene interactions play a crucial role in current theories of the architecture of complex traits, such as the omnigenic model [@doi:10.1016/j.cell.2017.05.038], which suggests that methods need to incorporate this complexity to disentangle disease-relevant mechanisms. Widespread gene pleiotropy, for instance, reveals the highly interconnected nature of transcriptional networks [@doi:10.1038/s41588-019-0481-0; @doi:10.1038/ng.3570], where potentially all genes expressed in disease-relevant cell types have a non-zero effect on the trait [@doi:10.1016/j.cell.2017.05.038; @doi:10.1016/j.cell.2019.04.014]. One way to learn these gene-gene interactions is using the concept of gene module: a group of genes with similar expression profiles across different conditions [@pmid:22955619; @pmid:25344726; @doi:10.1038/ng.3259]. In this context, several unsupervised approaches have been proposed to infer these gene-gene connections by extracting gene modules from co-expression patterns [@pm

In [46]:
par1 = process_paragraph(mod_section_paragraphs[3])
print(par1)

Gene-gene interactions are essential in current theories of complex traits, such as the omnigenic model (Boyle et al., 2017), which emphasizes the need for methods to account for this complexity in unraveling disease mechanisms. The widespread gene pleiotropy highlights the intricate interconnectedness of transcriptional networks (Watanabe et al., 2019; Finucane et al., 2015), suggesting that nearly all genes expressed in disease-relevant cell types have some impact on the trait (Boyle et al., 2017; Skene and Grant, 2019). One approach to understanding these interactions is through gene modules, which are groups of genes with similar expression patterns across various conditions (Langfelder and Horvath, 2008; Oldham et al., 2008; Zhang and Horvath, 2005). Several unsupervised methods have been proposed to uncover these gene-gene connections by identifying gene modules based on co-expression patterns (Eisen et al., 1998; Langfelder and Horvath, 2012; Tamayo et al., 1999). Matrix factori

In [47]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [48]:
display(paragraph_matches[-1])

('introduction',
 'These gene-gene interactions play a crucial role in current theories of the architecture of complex traits, such as the omnigenic model [@doi:10.1016/j.cell.2017.05.038], which suggests that methods need to incorporate this complexity to disentangle disease-relevant mechanisms. Widespread gene pleiotropy, for instance, reveals the highly interconnected nature of transcriptional networks [@doi:10.1038/s41588-019-0481-0; @doi:10.1038/ng.3570], where potentially all genes expressed in disease-relevant cell types have a non-zero effect on the trait [@doi:10.1016/j.cell.2017.05.038; @doi:10.1016/j.cell.2019.04.014]. One way to learn these gene-gene interactions is using the concept of gene module: a group of genes with similar expression profiles across different conditions [@pmid:22955619; @pmid:25344726; @doi:10.1038/ng.3259]. In this context, several unsupervised approaches have been proposed to infer these gene-gene connections by extracting gene modules from co-expre

####  Paragraph 03

In [49]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

Here we propose PhenoPLIER, an omnigenic approach that provides a gene module perspective to genetic studies. The flexibility of our method allows integrating different data modalities into the same representation for a joint analysis. We show that this module perspective can infer how groups of functionally-related genes influence complex traits, detect shared and distinct transcriptomic properties among traits, and predict how pharmacological perturbations affect genes' activity to exert their effects. PhenoPLIER maps gene-trait associations and drug-induced transcriptional responses into a common latent representation. For this, we integrate thousands of gene-trait associations (using TWAS from PhenomeXcan [@doi:10.1126/sciadv.aba2083]) and transcriptional profiles of drugs (from LINCS L1000 [@doi:10.1016/j.cell.2017.10.049]) into a low-dimensional space learned from public gene expression data on tens of thousands of RNA-seq samples (recount2 [@doi:10.1016/j.cels.2019.04.003; @doi:

In [50]:
par1 = process_paragraph(mod_section_paragraphs[4:8])
print(par1)

In this paper titled 'Projecting genetic associations through gene expression patterns highlights disease etiology and drug mechanisms,' we introduce PhenoPLIER, an omnigenic method that offers a gene module viewpoint for genetic studies. Our approach is versatile, allowing the integration of various data types into a unified representation for comprehensive analysis. Through the module perspective, we can deduce how clusters of functionally-related genes impact complex traits, identify shared and unique transcriptomic characteristics among traits, and predict the effects of pharmacological interventions on gene activity. PhenoPLIER aligns gene-trait associations and drug-induced transcriptional responses within a shared latent representation. To achieve this, we merge numerous gene-trait associations (via TWAS from PhenomeXcan) and drug transcriptional profiles (from LINCS L1000) into a low-dimensional space derived from public gene expression data from tens of thousands of RNA-seq sa

In [51]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [52]:
display(paragraph_matches[-1])

('introduction',
 "Here we propose PhenoPLIER, an omnigenic approach that provides a gene module perspective to genetic studies. The flexibility of our method allows integrating different data modalities into the same representation for a joint analysis. We show that this module perspective can infer how groups of functionally-related genes influence complex traits, detect shared and distinct transcriptomic properties among traits, and predict how pharmacological perturbations affect genes' activity to exert their effects. PhenoPLIER maps gene-trait associations and drug-induced transcriptional responses into a common latent representation. For this, we integrate thousands of gene-trait associations (using TWAS from PhenomeXcan [@doi:10.1126/sciadv.aba2083]) and transcriptional profiles of drugs (from LINCS L1000 [@doi:10.1016/j.cell.2017.10.049]) into a low-dimensional space learned from public gene expression data on tens of thousands of RNA-seq samples (recount2 [@doi:10.1016/j.cels

## Results (framework)

In [53]:
section_name = "results"

In [54]:
pr_filename = pr_files[2].filename
assert section_name in pr_filename
assert "framework" in pr_filename
print(pr_filename)

content/04.05.00.results_framework.md


### Original

In [55]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

### PhenoPLIER: an integration framework based on 


In [56]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

7

### Modified

In [57]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

### PhenoPLIER: an integration framework based on 


In [58]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

7

### Match

In [59]:
orig_section_paragraphs[0]

'### PhenoPLIER: an integration framework based on gene co-expression patterns'

In [60]:
mod_section_paragraphs[0]

'### PhenoPLIER: an integration framework based on gene co-expression patterns'

####  Paragraph 00

In [61]:
par0 = process_paragraph(orig_section_paragraphs[2])
print(par0)

PhenoPLIER is a flexible computational framework that combines gene-trait and gene-drug associations with gene modules expressed in specific contexts (Figure {@fig:entire_process}a). The approach uses a latent representation (with latent variables or LVs representing gene modules) derived from a large gene expression compendium (Figure {@fig:entire_process}b, top) to integrate TWAS with drug-induced transcriptional responses (Figure {@fig:entire_process}b, middle) for a joint analysis. The approach consists in three main components (Figure {@fig:entire_process}b, bottom, see [Methods](#sec:methods)): 1) an LV-based regression model to compute an association between an LV and a trait, 2) a clustering framework to learn groups of traits with shared transcriptomic properties, and 3) an LV-based drug repurposing approach that links diseases to potential treatments. We performed extensive simulations for our regression model ([Supplementary Note 1](#sm:reg:null_sim)) and clustering framewor

In [62]:
par1 = process_paragraph(mod_section_paragraphs[2])
print(par1)

PhenoPLIER is a computational framework that combines gene-trait and gene-drug associations with gene modules expressed in specific contexts. The approach utilizes a latent representation derived from a large gene expression compendium to integrate TWAS with drug-induced transcriptional responses for a joint analysis. The framework consists of three main components: an LV-based regression model to compute an association between an LV and a trait, a clustering framework to identify groups of traits with shared transcriptomic properties, and an LV-based drug repurposing approach that links diseases to potential treatments. Extensive simulations were performed for the regression model and clustering framework to ensure proper calibration and expected results under a model of no association.


In [63]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [64]:
display(paragraph_matches[-1])

('results',
 'PhenoPLIER is a flexible computational framework that combines gene-trait and gene-drug associations with gene modules expressed in specific contexts (Figure {@fig:entire_process}a). The approach uses a latent representation (with latent variables or LVs representing gene modules) derived from a large gene expression compendium (Figure {@fig:entire_process}b, top) to integrate TWAS with drug-induced transcriptional responses (Figure {@fig:entire_process}b, middle) for a joint analysis. The approach consists in three main components (Figure {@fig:entire_process}b, bottom, see [Methods](#sec:methods)): 1) an LV-based regression model to compute an association between an LV and a trait, 2) a clustering framework to learn groups of traits with shared transcriptomic properties, and 3) an LV-based drug repurposing approach that links diseases to potential treatments. We performed extensive simulations for our regression model ([Supplementary Note 1](#sm:reg:null_sim)) and clust

####  Paragraph 01

In [65]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

We used TWAS results from PhenomeXcan [@doi:10.1126/sciadv.aba2083] and the eMERGE network [@doi:10.1101/2021.10.21.21265225] as discovery and replication cohorts, respectively ([Methods](#sec:methods:twas)). PhenomeXcan provides gene-trait associations for 4,091 different diseases and traits from the UK Biobank [@doi:10.1038/s41586-018-0579-z] and other studies, whereas the analyses on eMERGE were performed across 309 phecodes. TWAS results were derived using two statistical methods (see [Methods](#sec:methods:predixcan)): 1) Summary-MultiXcan (S-MultiXcan) associations were used for the regression and clustering components, and 2) Summary-PrediXcan (S-PrediXcan) associations were used for the drug repurposing component. In addition, we also used colocalization results, which provide a probability of overlap between the GWAS and eQTL signals. For the drug-repurposing approach, we used transcriptional responses to small molecule perturbations from LINCS L1000 [@doi:10.1016/j.cell.2017.

In [66]:
par1 = process_paragraph(mod_section_paragraphs[3])
print(par1)

TWAS results from PhenomeXcan and the eMERGE network were used as discovery and replication cohorts, respectively. PhenomeXcan provided gene-trait associations for 4,091 diseases and traits from the UK Biobank and other studies, while eMERGE analyses were conducted across 309 phecodes. TWAS results utilized two statistical methods: Summary-MultiXcan (S-MultiXcan) associations for regression and clustering, and Summary-PrediXcan (S-PrediXcan) associations for drug repurposing. Colocalization results were also incorporated to assess overlap between GWAS and eQTL signals. For drug repurposing, transcriptional responses to small molecule perturbations from LINCS L1000 were utilized, encompassing 1,170 compounds.


In [67]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [68]:
display(paragraph_matches[-1])

('results',
 'We used TWAS results from PhenomeXcan [@doi:10.1126/sciadv.aba2083] and the eMERGE network [@doi:10.1101/2021.10.21.21265225] as discovery and replication cohorts, respectively ([Methods](#sec:methods:twas)). PhenomeXcan provides gene-trait associations for 4,091 different diseases and traits from the UK Biobank [@doi:10.1038/s41586-018-0579-z] and other studies, whereas the analyses on eMERGE were performed across 309 phecodes. TWAS results were derived using two statistical methods (see [Methods](#sec:methods:predixcan)): 1) Summary-MultiXcan (S-MultiXcan) associations were used for the regression and clustering components, and 2) Summary-PrediXcan (S-PrediXcan) associations were used for the drug repurposing component. In addition, we also used colocalization results, which provide a probability of overlap between the GWAS and eQTL signals. For the drug-repurposing approach, we used transcriptional responses to small molecule perturbations from LINCS L1000 [@doi:10.101

####  Paragraph 02

In [69]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

The latent gene expression representation was obtained from the MultiPLIER models [@doi:10.1016/j.cels.2019.04.003], which were derived by applying a matrix factorization method (the pathway-level information extractor or PLIER [@doi:10.1038/s41592-019-0456-1]) to recount2 [@doi:10.1038/nbt.3838] -- a uniformly-curated collection of transcript-level gene expression quantified by RNA-seq in a large, diverse set of samples collected across a range of disease states, cell types differentiation stages, and various stimuli (see [Methods](#sec:methods:multiplier)). The MultiPLIER models extracted 987 LVs by optimizing data reconstruction but also the alignment of LVs with prior knowledge/pathways.


In [70]:
par1 = process_paragraph(mod_section_paragraphs[4])
print(par1)

The latent gene expression representation was obtained from the MultiPLIER models, which were derived using a matrix factorization method called the pathway-level information extractor (PLIER) applied to recount2. Recount2 is a uniformly-curated collection of transcript-level gene expression quantified by RNA-seq in a large, diverse set of samples collected across different disease states, cell types, differentiation stages, and stimuli. The MultiPLIER models extracted 987 latent variables (LVs) by optimizing data reconstruction and aligning LVs with prior knowledge and pathways.


In [71]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [72]:
display(paragraph_matches[-1])

('results',
 'The latent gene expression representation was obtained from the MultiPLIER models [@doi:10.1016/j.cels.2019.04.003], which were derived by applying a matrix factorization method (the pathway-level information extractor or PLIER [@doi:10.1038/s41592-019-0456-1]) to recount2 [@doi:10.1038/nbt.3838] -- a uniformly-curated collection of transcript-level gene expression quantified by RNA-seq in a large, diverse set of samples collected across a range of disease states, cell types differentiation stages, and various stimuli (see [Methods](#sec:methods:multiplier)). The MultiPLIER models extracted 987 LVs by optimizing data reconstruction but also the alignment of LVs with prior knowledge/pathways.',
 'The latent gene expression representation was obtained from the MultiPLIER models, which were derived using a matrix factorization method called the pathway-level information extractor (PLIER) applied to recount2. Recount2 is a uniformly-curated collection of transcript-level gene

####  Paragraph 03

In [73]:
par0 = process_paragraph(orig_section_paragraphs[5])
print(par0)

Each LV or gene module represents a group of weighted genes expressed together in the same tissues and cell types as a functional unit. Since LVs might represent a functional set of genes regulated by the same transcriptional program [@doi:10.1186/1471-2164-7-187; @doi:10.1186/s13059-019-1835-8], we conjecture that the projection of TWAS and pharmacologic perturbations data into this latent space could provide a better mechanistic understanding. For this projection of different data modalities into the same space, PhenoPLIER converts gene associations to an LV score: all genes' standardized effect sizes for a trait (from TWAS) or differential expression values for a drug (from pharmacologic perturbation data) are multiplied by the LV genes' weights and summed, producing a single value. Instead of looking at individual genes, this process links different traits and drugs to functionally-related groups of genes or LVs. PhenoPLIER uses LVs annotations about the specific conditions where t

In [74]:
par1 = process_paragraph(mod_section_paragraphs[5])
print(par1)

Each latent variable (LV) or gene module represents a group of weighted genes expressed together in the same tissues and cell types as a functional unit. LVs may represent a set of genes regulated by the same transcriptional program, allowing for a better mechanistic understanding when projecting TWAS and pharmacologic perturbations data into this latent space. PhenoPLIER converts gene associations to an LV score by multiplying all genes' standardized effect sizes for a trait (from TWAS) or differential expression values for a drug (from pharmacologic perturbation data) by the LV genes' weights and summing them to produce a single value. This process links different traits and drugs to functionally-related groups of genes or LVs, with annotations about the specific conditions where the group of genes is expressed, such as cell types, tissues, disease stages, or developmental stages, enhancing interpretability. MultiPLIER's models provide this information by linking LVs to samples annot

In [75]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [76]:
display(paragraph_matches[-1])

('results',
 "Each LV or gene module represents a group of weighted genes expressed together in the same tissues and cell types as a functional unit. Since LVs might represent a functional set of genes regulated by the same transcriptional program [@doi:10.1186/1471-2164-7-187; @doi:10.1186/s13059-019-1835-8], we conjecture that the projection of TWAS and pharmacologic perturbations data into this latent space could provide a better mechanistic understanding. For this projection of different data modalities into the same space, PhenoPLIER converts gene associations to an LV score: all genes' standardized effect sizes for a trait (from TWAS) or differential expression values for a drug (from pharmacologic perturbation data) are multiplied by the LV genes' weights and summed, producing a single value. Instead of looking at individual genes, this process links different traits and drugs to functionally-related groups of genes or LVs. PhenoPLIER uses LVs annotations about the specific cond

####  Paragraph 04

In [77]:
par0 = process_paragraph(orig_section_paragraphs[6])
print(par0)

Therefore, PhenoPLIER allows the user to address specific questions, namely: do disease-associated genes belong to modules expressed in specific tissues and cell types? Are these cell type-specific modules associated with _different_ diseases, thus potentially representing a "network pleiotropy" example from an omnigenic point of view [@doi:10.1016/j.cell.2017.05.038]? Is there a subset of module's genes that is closer to the definition of "core" genes (i.e., directly affecting the trait with no mediated regulation of other genes [@doi:10.1016/j.cell.2019.04.014]) and thus represents alternative and potentially better candidate targets? Are drugs perturbing these transcriptional mechanisms, and can they suggest potential mechanisms of action?


In [78]:
par1 = process_paragraph(mod_section_paragraphs[6])
print(par1)

PhenoPLIER enables users to explore whether disease-associated genes are part of modules expressed in specific tissues and cell types. These cell type-specific modules may be linked to various diseases, demonstrating a concept of "network pleiotropy" from an omnigenic perspective. Additionally, PhenoPLIER can identify a subset of genes within modules that directly impact traits, known as "core" genes, which could serve as alternative and potentially more effective therapeutic targets. Furthermore, the tool can identify drugs that perturb these transcriptional mechanisms, providing insights into potential mechanisms of action.


In [79]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [80]:
display(paragraph_matches[-1])

('results',
 'Therefore, PhenoPLIER allows the user to address specific questions, namely: do disease-associated genes belong to modules expressed in specific tissues and cell types? Are these cell type-specific modules associated with _different_ diseases, thus potentially representing a "network pleiotropy" example from an omnigenic point of view [@doi:10.1016/j.cell.2017.05.038]? Is there a subset of module\'s genes that is closer to the definition of "core" genes (i.e., directly affecting the trait with no mediated regulation of other genes [@doi:10.1016/j.cell.2019.04.014]) and thus represents alternative and potentially better candidate targets? Are drugs perturbing these transcriptional mechanisms, and can they suggest potential mechanisms of action?',
 'PhenoPLIER enables users to explore whether disease-associated genes are part of modules expressed in specific tissues and cell types. These cell type-specific modules may be linked to various diseases, demonstrating a concept o

## Results (CRISPR)

In [81]:
# section_name = "results"

In [82]:
pr_filename = pr_files[3].filename
assert section_name in pr_filename
assert "crispr" in pr_filename
print(pr_filename)

content/04.05.01.results_crispr.md


### Original

In [83]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

### LVs link genes that alter lipid accumulation w


In [84]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

6

### Modified

In [85]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

### LVs link genes that alter lipid accumulation w


In [86]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

6

### Match

In [87]:
orig_section_paragraphs[0]

'### LVs link genes that alter lipid accumulation with relevant traits and tissues'

In [88]:
mod_section_paragraphs[0]

'### LVs link genes that alter lipid accumulation with relevant traits and tissues'

####  Paragraph 00

In [89]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

Our first experiment attempted to answer whether genes in a disease-relevant LV could represent potential therapeutic targets. For this, the first step was to obtain a set of genes strongly associated with a phenotype of interest. Therefore, we performed a fluorescence-based CRISPR-Cas9 in the HepG2 cell line and identified 462 genes associated with lipid regulation ([Methods](#sec:methods:crispr)). From these, we selected two high-confidence gene sets that either caused a decrease or increase of lipids: a lipids-decreasing gene-set with eight genes: *BLCAP*, *FBXW7*, *INSIG2*, *PCYT2*, *PTEN*, *SOX9*, *TCF7L2*, *UBE2J2*; and a lipids-increasing gene-set with six genes: *ACACA*, *DGAT2*, *HILPDA*, *MBTPS1*, *SCAP*, *SRPR* (Supplementary Data 2).


In [90]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

In our first experiment, we aimed to determine if genes in a disease-relevant LV could serve as potential therapeutic targets. To do this, we utilized a fluorescence-based CRISPR-Cas9 approach in the HepG2 cell line, identifying 462 genes associated with lipid regulation. From this pool, we selected two high-confidence gene sets: one that led to a decrease in lipids, consisting of eight genes (*BLCAP*, *FBXW7*, *INSIG2*, *PCYT2*, *PTEN*, *SOX9*, *TCF7L2*, *UBE2J2*), and another that caused an increase in lipids, comprised of six genes (*ACACA*, *DGAT2*, *HILPDA*, *MBTPS1*, *SCAP*, *SRPR*) (Supplementary Data 2). This selection process is detailed in the Methods section.


In [91]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [92]:
display(paragraph_matches[-1])

('results',
 'Our first experiment attempted to answer whether genes in a disease-relevant LV could represent potential therapeutic targets. For this, the first step was to obtain a set of genes strongly associated with a phenotype of interest. Therefore, we performed a fluorescence-based CRISPR-Cas9 in the HepG2 cell line and identified 462 genes associated with lipid regulation ([Methods](#sec:methods:crispr)). From these, we selected two high-confidence gene sets that either caused a decrease or increase of lipids: a lipids-decreasing gene-set with eight genes: *BLCAP*, *FBXW7*, *INSIG2*, *PCYT2*, *PTEN*, *SOX9*, *TCF7L2*, *UBE2J2*; and a lipids-increasing gene-set with six genes: *ACACA*, *DGAT2*, *HILPDA*, *MBTPS1*, *SCAP*, *SRPR* (Supplementary Data 2).',
 'In our first experiment, we aimed to determine if genes in a disease-relevant LV could serve as potential therapeutic targets. To do this, we utilized a fluorescence-based CRISPR-Cas9 approach in the HepG2 cell line, identifyi

####  Paragraph 01

In [93]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

Next, we analyzed all 987 LVs using Fast Gene Set Enrichment Analysis (FGSEA) [@doi:10.1101/060012], and found 15 LVs nominally enriched (unadjusted *P* < 0.01) with these lipid-altering gene-sets (Tables @tbl:sup:lipids_crispr:modules_enriched_increase and @tbl:sup:lipids_crispr:modules_enriched_decrease). Among those with reliable sample metadata, LV246, the top LV associated with the lipids-increasing gene-set, contained genes mainly co-expressed in adipose tissue (Figure {@fig:lv246}a), which plays a key role in coordinating and regulating lipid metabolism. Using our regression framework across all traits in PhenomeXcan, we found that gene weights for this LV were predictive of gene associations for plasma lipids, high cholesterol, and Alzheimer's disease (Table @tbl:sup:phenomexcan_assocs:lv246, FDR < 1e-23). These lipids-related associations also replicated across the 309 traits in eMERGE (Table @tbl:sup:emerge_assocs:lv246), where LV246 was significantly associated with hypercho

In [94]:
par1 = process_paragraph(mod_section_paragraphs[3])
print(par1)

We analyzed 987 Latent Variables (LVs) using Fast Gene Set Enrichment Analysis (FGSEA), identifying 15 LVs enriched with lipid-altering gene sets at a nominal significance level (P < 0.01). Among these, LV246 stood out as the top LV associated with genes co-expressed in adipose tissue, crucial for lipid metabolism regulation. Regression analysis in PhenomeXcan revealed that gene weights in LV246 were predictive of gene associations for plasma lipids, high cholesterol, and Alzheimer's disease. These associations were replicated across 309 traits in eMERGE, where LV246 showed significant links to hypercholesterolemia, hyperlipidemia, and disorders of lipoid metabolism.


In [95]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [96]:
display(paragraph_matches[-1])

('results',
 "Next, we analyzed all 987 LVs using Fast Gene Set Enrichment Analysis (FGSEA) [@doi:10.1101/060012], and found 15 LVs nominally enriched (unadjusted *P* < 0.01) with these lipid-altering gene-sets (Tables @tbl:sup:lipids_crispr:modules_enriched_increase and @tbl:sup:lipids_crispr:modules_enriched_decrease). Among those with reliable sample metadata, LV246, the top LV associated with the lipids-increasing gene-set, contained genes mainly co-expressed in adipose tissue (Figure {@fig:lv246}a), which plays a key role in coordinating and regulating lipid metabolism. Using our regression framework across all traits in PhenomeXcan, we found that gene weights for this LV were predictive of gene associations for plasma lipids, high cholesterol, and Alzheimer's disease (Table @tbl:sup:phenomexcan_assocs:lv246, FDR < 1e-23). These lipids-related associations also replicated across the 309 traits in eMERGE (Table @tbl:sup:emerge_assocs:lv246), where LV246 was significantly associated

####  Paragraph 02

In [97]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

Two high-confidence genes from our CRISPR screening, *DGAT2* and *ACACA*, are responsible for encoding enzymes for triglycerides and fatty acid synthesis and were among the highest-weighted genes of LV246 (Figure {@fig:lv246}b, in boldface). However, in contrast to other members of LV246, *DGAT2* and *ACACA* were not associated nor colocalized with any of the cardiovascular-related traits and thus would not have been prioritized by TWAS alone; instead, other members of LV246, such as *SCD*, *LPL*, *FADS2*, *HMGCR*, and *LDLR*, were significantly associated and colocalized with lipid-related traits. This lack of association of two high-confidence genes from our CRISPR screen might be explained from an omnigenic point of view [@doi:10.1016/j.cell.2019.04.014]. Assuming that the TWAS models for *DGAT2* and *ACACA* capture all common *cis*-eQTLs (the only genetic component of gene expression that TWAS can capture) and there are no rare *cis*-eQTLs, these two genes might represent "core" ge

In [98]:
par1 = process_paragraph(mod_section_paragraphs[4])
print(par1)

Two high-confidence genes identified in our CRISPR screening, DGAT2 and ACACA, encode enzymes for triglycerides and fatty acid synthesis and were among the highest-weighted genes of LV246 (Figure 2b). However, unlike other genes in LV246, DGAT2 and ACACA were not associated with any cardiovascular-related traits. Instead, genes like SCD, LPL, FADS2, HMGCR, and LDLR in LV246 showed significant associations with lipid-related traits. This lack of association for DGAT2 and ACACA may be explained by the omnigenic theory. If the TWAS models for DGAT2 and ACACA capture all common cis-eQTLs and there are no rare cis-eQTLs, these genes may be considered "core" genes directly affecting the trait without regulation from other genes, while the other genes in LV246 may be "peripheral" genes that regulate them.


In [99]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [100]:
display(paragraph_matches[-1])

('results',
 'Two high-confidence genes from our CRISPR screening, *DGAT2* and *ACACA*, are responsible for encoding enzymes for triglycerides and fatty acid synthesis and were among the highest-weighted genes of LV246 (Figure {@fig:lv246}b, in boldface). However, in contrast to other members of LV246, *DGAT2* and *ACACA* were not associated nor colocalized with any of the cardiovascular-related traits and thus would not have been prioritized by TWAS alone; instead, other members of LV246, such as *SCD*, *LPL*, *FADS2*, *HMGCR*, and *LDLR*, were significantly associated and colocalized with lipid-related traits. This lack of association of two high-confidence genes from our CRISPR screen might be explained from an omnigenic point of view [@doi:10.1016/j.cell.2019.04.014]. Assuming that the TWAS models for *DGAT2* and *ACACA* capture all common *cis*-eQTLs (the only genetic component of gene expression that TWAS can capture) and there are no rare *cis*-eQTLs, these two genes might repre

## Results (drug-disease prediction)

In [101]:
# section_name = "results"

In [102]:
pr_filename = pr_files[4].filename
assert section_name in pr_filename
assert "drug_disease" in pr_filename
print(pr_filename)

content/04.15.results_drug_disease_prediction.md


### Original

In [103]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

### LVs predict drug-disease pairs better than sin


In [104]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

11

### Modified

In [105]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

### LVs predict drug-disease pairs better than sin


In [106]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

11

### Match

In [107]:
orig_section_paragraphs[0]

'### LVs predict drug-disease pairs better than single genes'

In [108]:
mod_section_paragraphs[0]

'### LVs predict drug-disease pairs better than single genes'

####  Paragraph 00

In [109]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

We next determined how substituting LVs for individual genes predicted known treatment-disease relationships. For this, we used the transcriptional responses to small molecule perturbations profiled in LINCS L1000 [@doi:10.1016/j.cell.2017.10.049], which were further processed and mapped to DrugBank IDs [@doi:10.1093/nar/gkt1068; @doi:10.7554/eLife.26726; @doi:10.5281/zenodo.47223]. Based on an established drug repurposing strategy that matches reversed transcriptome patterns between genes and drug-induced perturbations [@doi:10.1126/scitranslmed.3002648; @doi:10.1126/scitranslmed.3001318], we adopted a previously described framework that uses imputed transcriptomes from TWAS to prioritize drug candidates [@doi:10.1038/nn.4618]. For this, we computed a drug-disease score by calculating the negative dot product between the $z$-scores for a disease (from TWAS) and the $z$-scores for a drug (from LINCS) across sets of genes of different sizes (see [Methods](#sec:methods:drug)). Therefore,

In [110]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

We assessed the ability of substituting latent variables (LVs) for individual genes to predict established treatment-disease relationships by analyzing the transcriptional responses to small molecule perturbations from LINCS L1000. These responses were processed and linked to DrugBank IDs. By applying a drug repurposing strategy that matches reversed transcriptome patterns between genes and drug-induced perturbations, we utilized imputed transcriptomes from TWAS to prioritize drug candidates. A drug-disease score was computed by comparing the $z$-scores for a disease (from TWAS) and a drug (from LINCS) across different gene sets. A high score for a drug-disease pair indicated potential treatment efficacy based on the drug's regulation of disease-associated gene expression. Additionally, we projected drug expression profiles into our latent representation to estimate the impact of pharmacological perturbations on gene module activity. The prediction performance was evaluated using a cur

In [111]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [112]:
display(paragraph_matches[-1])

('results',
 'We next determined how substituting LVs for individual genes predicted known treatment-disease relationships. For this, we used the transcriptional responses to small molecule perturbations profiled in LINCS L1000 [@doi:10.1016/j.cell.2017.10.049], which were further processed and mapped to DrugBank IDs [@doi:10.1093/nar/gkt1068; @doi:10.7554/eLife.26726; @doi:10.5281/zenodo.47223]. Based on an established drug repurposing strategy that matches reversed transcriptome patterns between genes and drug-induced perturbations [@doi:10.1126/scitranslmed.3002648; @doi:10.1126/scitranslmed.3001318], we adopted a previously described framework that uses imputed transcriptomes from TWAS to prioritize drug candidates [@doi:10.1038/nn.4618]. For this, we computed a drug-disease score by calculating the negative dot product between the $z$-scores for a disease (from TWAS) and the $z$-scores for a drug (from LINCS) across sets of genes of different sizes (see [Methods](#sec:methods:drug

####  Paragraph 01

In [113]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

It is important to note that the gene-trait associations and drug-induced expression profiles projected into the latent space represent a compressed version of the entire set of results. Despite this information loss, the LV-based method outperformed the gene-based one with an area under the curve of 0.632 and an average precision of 0.858 (Figure @fig:drug_disease:roc_pr). The prediction results suggested that this low-dimensional space captures biologically meaningful patterns that can link pathophysiological processes with the mechanism of action of drugs.


In [114]:
par1 = process_paragraph(mod_section_paragraphs[3])
print(par1)

It is important to note that the gene-trait associations and drug-induced expression profiles projected into the latent space represent a condensed summary of the results. Despite this simplification, the LV-based method performed better than the gene-based method, with an area under the curve of 0.632 and an average precision of 0.858 (Figure 1). The prediction results indicated that this reduced-dimensional space captures meaningful biological patterns that connect pathophysiological processes with drug mechanisms.


In [115]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [116]:
display(paragraph_matches[-1])

('results',
 'It is important to note that the gene-trait associations and drug-induced expression profiles projected into the latent space represent a compressed version of the entire set of results. Despite this information loss, the LV-based method outperformed the gene-based one with an area under the curve of 0.632 and an average precision of 0.858 (Figure @fig:drug_disease:roc_pr). The prediction results suggested that this low-dimensional space captures biologically meaningful patterns that can link pathophysiological processes with the mechanism of action of drugs.',
 'It is important to note that the gene-trait associations and drug-induced expression profiles projected into the latent space represent a condensed summary of the results. Despite this simplification, the LV-based method performed better than the gene-based method, with an area under the curve of 0.632 and an average precision of 0.858 (Figure 1). The prediction results indicated that this reduced-dimensional spa

####  Paragraph 02

In [117]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

We examined a specific drug-disease pair to determine whether the LVs driving the prediction were biologically plausible. Nicotinic acid (niacin) is a B vitamin widely used clinically to treat lipid disorders, although there is controversy on its clinical utility in preventing cardiovascular disease [@pmid:22085343; @pmid:25014686; @pmid:30977858]. Niacin exerts its effects on multiple tissues, although its mechanisms are not well understood [@doi:10.1016/j.amjcard.2008.02.029; @doi:10.1194/jlr.S092007; @pmid:24363242; @pmid:24713591]. This compound can increase high-density lipoprotein (HDL) by inhibiting an HDL catabolism receptor in the liver. Niacin also inhibits diacylglycerol acyltransferase–2 (DGAT2), which decreases the production of low-density lipoproteins (LDL) either by modulating triglyceride synthesis in hepatocytes or by inhibiting adipocyte triglyceride lipolysis [@doi:10.1016/j.amjcard.2008.02.029]. Niacin was one of the drugs in the gold standard set indicated for ath

In [118]:
par1 = process_paragraph(mod_section_paragraphs[4])
print(par1)

We investigated the relationship between niacin and coronary artery disease (CAD) using gene expression patterns. Niacin, a B vitamin commonly used to treat lipid disorders, has unclear clinical benefits in preventing cardiovascular disease. Its mechanisms of action involve inhibiting an HDL catabolism receptor in the liver and DGAT2, which reduces LDL production. Our analysis showed that niacin was predicted as a potential treatment for CAD and atherosclerosis (AT) using gene-based and latent variable (LV)-based methods. LV246, associated with plasma lipids and high cholesterol, was identified as a key factor in predicting niacin's therapeutic effects on AT. Additionally, LV246 was also relevant for ischaemic heart disease and high cholesterol.


In [119]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [120]:
display(paragraph_matches[-1])

('results',
 'We examined a specific drug-disease pair to determine whether the LVs driving the prediction were biologically plausible. Nicotinic acid (niacin) is a B vitamin widely used clinically to treat lipid disorders, although there is controversy on its clinical utility in preventing cardiovascular disease [@pmid:22085343; @pmid:25014686; @pmid:30977858]. Niacin exerts its effects on multiple tissues, although its mechanisms are not well understood [@doi:10.1016/j.amjcard.2008.02.029; @doi:10.1194/jlr.S092007; @pmid:24363242; @pmid:24713591]. This compound can increase high-density lipoprotein (HDL) by inhibiting an HDL catabolism receptor in the liver. Niacin also inhibits diacylglycerol acyltransferase–2 (DGAT2), which decreases the production of low-density lipoproteins (LDL) either by modulating triglyceride synthesis in hepatocytes or by inhibiting adipocyte triglyceride lipolysis [@doi:10.1016/j.amjcard.2008.02.029]. Niacin was one of the drugs in the gold standard set ind

####  Paragraph 03

In [121]:
par0 = process_paragraph(orig_section_paragraphs[7])
print(par0)

The analysis of other top niacin-contributing LVs across different cardiovascular diseases revealed additional mechanisms of action. For example, *GPR109A/HCAR2* encodes a G protein-coupled high-affinity niacin receptor in adipocytes and immune cells, including monocytes, macrophages, neutrophils and dendritic cells [@doi:10.1016/j.tips.2006.05.008; @doi:10.1038/sj.jid.5700586]. It was initially thought that the antiatherogenic effects of niacin were solely due to the inhibition of lipolysis in adipose tissue. However, it has been shown that nicotinic acid can reduce atherosclerosis progression independently of its antidyslipidemic activity by activating *GPR109A* in immune cells [@doi:10.1172/JCI41651], thus boosting anti-inflammatory processes [@doi:10.1161/ATVBAHA.108.179283]. In addition, flushing, a common adverse effect of niacin, is also produced by the activation of GPR109A in Langerhans cells (macrophages of the skin). This alternative mechanism for niacin could have been hypo

In [122]:
par1 = process_paragraph(mod_section_paragraphs[7])
print(par1)

Analysis of top niacin-contributing LVs in various cardiovascular diseases revealed additional mechanisms of action. For instance, the gene *GPR109A/HCAR2* encodes a high-affinity niacin receptor in adipocytes and immune cells, including monocytes, macrophages, neutrophils, and dendritic cells. Initially, niacin's antiatherogenic effects were attributed to inhibiting lipolysis in adipose tissue. However, studies have shown that nicotinic acid can independently reduce atherosclerosis progression by activating *GPR109A* in immune cells, promoting anti-inflammatory processes. Moreover, flushing, a common niacin side effect, is also triggered by *GPR109A* activation in Langerhans cells. This alternative mechanism could have been predicted by examining cell types expressing top-contributing modules, such as LV116 and LV931, which show strong signatures in immune cells like monocytes and macrophages (Figures @fig:lv116:cell_types and @fig:sup:lv931, Tables @tbl:sup:multiplier_pathways:lv116 

In [123]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [124]:
display(paragraph_matches[-1])

('results',
 "The analysis of other top niacin-contributing LVs across different cardiovascular diseases revealed additional mechanisms of action. For example, *GPR109A/HCAR2* encodes a G protein-coupled high-affinity niacin receptor in adipocytes and immune cells, including monocytes, macrophages, neutrophils and dendritic cells [@doi:10.1016/j.tips.2006.05.008; @doi:10.1038/sj.jid.5700586]. It was initially thought that the antiatherogenic effects of niacin were solely due to the inhibition of lipolysis in adipose tissue. However, it has been shown that nicotinic acid can reduce atherosclerosis progression independently of its antidyslipidemic activity by activating *GPR109A* in immune cells [@doi:10.1172/JCI41651], thus boosting anti-inflammatory processes [@doi:10.1161/ATVBAHA.108.179283]. In addition, flushing, a common adverse effect of niacin, is also produced by the activation of GPR109A in Langerhans cells (macrophages of the skin). This alternative mechanism for niacin could 

####  Paragraph 04

In [125]:
par0 = process_paragraph(orig_section_paragraphs[10])
print(par0)

Beyond cardiovascular traits, there are other potentially interesting LVs that could extend our understanding of the mechanisms of niacin. For example, LV66, one of the top LVs affected by niacin (Figure @fig:sup:lv66), was mainly expressed in ovarian granulosa cells. This compound has been very recently considered a potential therapeutic for ovarian diseases [@doi:10.1159/000495051; @doi:10.1071/RD20306], as it was found to promote follicle growth and inhibit granulosa cell apoptosis in animal models.


In [126]:
par1 = process_paragraph(mod_section_paragraphs[10])
print(par1)

In addition to cardiovascular traits, other LVs could enhance our knowledge of niacin mechanisms. For instance, LV66, a key LV impacted by niacin (Figure 1), showed predominant expression in ovarian granulosa cells. Recent studies have suggested niacin as a potential therapy for ovarian diseases, as it has been shown to stimulate follicle growth and prevent granulosa cell apoptosis in animal models.


In [127]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [128]:
display(paragraph_matches[-1])

('results',
 'Beyond cardiovascular traits, there are other potentially interesting LVs that could extend our understanding of the mechanisms of niacin. For example, LV66, one of the top LVs affected by niacin (Figure @fig:sup:lv66), was mainly expressed in ovarian granulosa cells. This compound has been very recently considered a potential therapeutic for ovarian diseases [@doi:10.1159/000495051; @doi:10.1071/RD20306], as it was found to promote follicle growth and inhibit granulosa cell apoptosis in animal models.',
 'In addition to cardiovascular traits, other LVs could enhance our knowledge of niacin mechanisms. For instance, LV66, a key LV impacted by niacin (Figure 1), showed predominant expression in ovarian granulosa cells. Recent studies have suggested niacin as a potential therapy for ovarian diseases, as it has been shown to stimulate follicle growth and prevent granulosa cell apoptosis in animal models.')

## Results (clustering)

In [129]:
# section_name = "results"

In [130]:
pr_filename = pr_files[5].filename
assert section_name in pr_filename
assert "traits_clustering" in pr_filename
print(pr_filename)

content/04.20.00.results_traits_clustering.md


### Original

In [131]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

### LVs reveal trait clusters with shared transcri


In [132]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

9

### Modified

In [133]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

### LVs reveal trait clusters with shared transcri


In [134]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

10

### Match

In [135]:
orig_section_paragraphs[0]

'### LVs reveal trait clusters with shared transcriptomic properties'

In [136]:
mod_section_paragraphs[0]

'### LVs reveal trait clusters with shared transcriptomic properties'

####  Paragraph 00

In [137]:
par0 = process_paragraph(orig_section_paragraphs[2])
print(par0)

We used the projection of gene-trait associations into the latent space to find groups of clusters linked by the same transcriptional processes. Since individual clustering algorithms have different biases (i.e., assumptions about the data structure), we designed a consensus clustering framework that combines solutions or partitions of traits generated by different methods ([Methods](#sec:methods:clustering)). Consensus or ensemble approaches have been recommended to avoid several pitfalls when performing cluster analysis on biological data [@doi:10.1126/scisignal.aad1932]. Since diversity in the ensemble is crucial for these methods, we generated different data versions which were processed using different methods with varying sets of parameters (Figure {@fig:clustering:design}a). Then, a consensus function combines the ensemble into a consolidated solution, which has been shown to outperform any individual member of the ensemble [@Strehl2002; @doi:10.1109/TPAMI.2005.113]. Our cluster

In [138]:
par1 = process_paragraph(mod_section_paragraphs[2])
print(par1)

Gene-trait associations were projected into a latent space to identify clusters with similar transcriptional processes. To address biases in individual clustering algorithms, we developed a consensus clustering framework that combines trait partitions from different methods. This approach, recommended for biological data analysis, involved generating diverse data versions processed with varying parameters. A consensus function then consolidated these solutions, outperforming individual ensemble members. Our pipeline produced 15 final consensus clustering solutions, with cluster numbers ranging from 5 to 29. Instead of selecting a specific solution, we utilized a clustering tree to identify stable trait groups across various resolutions. To determine the distinguishing latent variables, a decision tree classifier was trained on the input data using the identified clusters as labels.


In [139]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [140]:
display(paragraph_matches[-1])

('results',
 'We used the projection of gene-trait associations into the latent space to find groups of clusters linked by the same transcriptional processes. Since individual clustering algorithms have different biases (i.e., assumptions about the data structure), we designed a consensus clustering framework that combines solutions or partitions of traits generated by different methods ([Methods](#sec:methods:clustering)). Consensus or ensemble approaches have been recommended to avoid several pitfalls when performing cluster analysis on biological data [@doi:10.1126/scisignal.aad1932]. Since diversity in the ensemble is crucial for these methods, we generated different data versions which were processed using different methods with varying sets of parameters (Figure {@fig:clustering:design}a). Then, a consensus function combines the ensemble into a consolidated solution, which has been shown to outperform any individual member of the ensemble [@Strehl2002; @doi:10.1109/TPAMI.2005.113

####  Paragraph 01

In [141]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

We found that phenotypes were grouped into five clear branches, defined by their first node at the top of the Figure @fig:clustering:tree: 0) a "large" branch that includes most of the traits subdivided only starting at $k$=16 (with asthma, subjective well-being traits, and nutrient intake clusters), 1) heel bone-densitometry measurements, 2) hematological assays on red blood cells, 3) physical measures, including spirometry and body impedance, and anthropometric traits with fat-free and fat mass measures in separate sub-branches, and 4) a "complex" branch including keratometry measurements, assays on white blood cells and platelets, skin and hair color traits, autoimmune disorders, and cardiovascular diseases (which also included other cardiovascular-related traits such as hand-grip strength [@pmid:25982160], and environmental/behavioral factors such as physical activity and diet) (see Supplementary Data 3-7 for all clustering results). Within these branches, results were relatively s

In [142]:
par1 = process_paragraph(mod_section_paragraphs[4])
print(par1)

Phenotypes were grouped into five branches based on the clustering analysis. The first branch, labeled as "large," encompassed most traits and subdivided only at $k$=16, including asthma, subjective well-being traits, and nutrient intake clusters. The second branch included heel bone-densitometry measurements, while the third branch consisted of hematological assays on red blood cells, physical measures such as spirometry and body impedance, and anthropometric traits with fat-free and fat mass measures in separate sub-branches. The fourth branch, labeled as "complex," comprised keratometry measurements, assays on white blood cells and platelets, skin and hair color traits, autoimmune disorders, and cardiovascular diseases, along with other cardiovascular-related traits like hand-grip strength and environmental/behavioral factors such as physical activity and diet. Results within these branches were stable, with the same traits often clustered together across different resolutions. Arro

In [143]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [144]:
display(paragraph_matches[-1])

('results',
 'We found that phenotypes were grouped into five clear branches, defined by their first node at the top of the Figure @fig:clustering:tree: 0) a "large" branch that includes most of the traits subdivided only starting at $k$=16 (with asthma, subjective well-being traits, and nutrient intake clusters), 1) heel bone-densitometry measurements, 2) hematological assays on red blood cells, 3) physical measures, including spirometry and body impedance, and anthropometric traits with fat-free and fat mass measures in separate sub-branches, and 4) a "complex" branch including keratometry measurements, assays on white blood cells and platelets, skin and hair color traits, autoimmune disorders, and cardiovascular diseases (which also included other cardiovascular-related traits such as hand-grip strength [@pmid:25982160], and environmental/behavioral factors such as physical activity and diet) (see Supplementary Data 3-7 for all clustering results). Within these branches, results wer

####  Paragraph 02

In [145]:
par0 = process_paragraph(orig_section_paragraphs[6])
print(par0)

Next, we analyzed which LVs were driving these clusters of traits. For this, we trained decision tree classifiers on the input data using each cluster at $k$=29 (bottom of Figure @fig:clustering:tree) as labels (see [Methods](#sec:methods:clustering)). This procedure yielded the top LVs that were most discriminative for each cluster. Several of these LVs were well-aligned to existing pathways (Figure @fig:clustering:heatmap), whereas others were not aligned to prior knowledge but still expressed in relevant tissues (Figure @fig:sup:clustering:novel:heatmap). In Figure @fig:clustering:heatmap, it can be seen that some LVs were highly specific to certain traits, while others were associated with a wide range of different phenotypes, thus potentially involved in more general biological functions. We used our regression framework to determine whether these LVs were significantly associated with different traits. For example, LVs such as LV928 and LV30, which were well-aligned to early prog

In [146]:
par1 = process_paragraph(mod_section_paragraphs[6:8])
print(par1)

We identified the key latent variables (LVs) driving clusters of traits by training decision tree classifiers on the input data using each cluster at $k$=29 as labels. This analysis is shown in Figure 1 at the bottom of the clustering tree. The top LVs that were most discriminative for each cluster were determined through this process. Some of these LVs aligned well with known pathways, as depicted in Figure 2, while others were not previously associated with known pathways but were expressed in relevant tissues, as shown in Figure 3. In Figure 2, it is evident that certain LVs were specific to particular traits, while others were linked to a broad range of phenotypes, potentially indicating involvement in general biological functions. Using our regression framework, we assessed the significant associations of these LVs with various traits. For instance, LVs like LV928 and LV30, which aligned with early progenitors of the erythrocytes lineage, were predominantly expressed in early diff

In [147]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [148]:
display(paragraph_matches[-1])

('results',
 'Next, we analyzed which LVs were driving these clusters of traits. For this, we trained decision tree classifiers on the input data using each cluster at $k$=29 (bottom of Figure @fig:clustering:tree) as labels (see [Methods](#sec:methods:clustering)). This procedure yielded the top LVs that were most discriminative for each cluster. Several of these LVs were well-aligned to existing pathways (Figure @fig:clustering:heatmap), whereas others were not aligned to prior knowledge but still expressed in relevant tissues (Figure @fig:sup:clustering:novel:heatmap). In Figure @fig:clustering:heatmap, it can be seen that some LVs were highly specific to certain traits, while others were associated with a wide range of different phenotypes, thus potentially involved in more general biological functions. We used our regression framework to determine whether these LVs were significantly associated with different traits. For example, LVs such as LV928 and LV30, which were well-aligned

####  Paragraph 03

In [149]:
par0 = process_paragraph(orig_section_paragraphs[7])
print(par0)

The sub-branches of autoimmune and cardiovascular diseases merged together at $k=10$ (middle of Figure @fig:clustering:tree), so we expected to find LVs that specifically affect one or both of these types of diseases. For example, LV57, expressed in T cells (Figure @fig:sup:lv57 and Table @tbl:sup:multiplier_pathways:lv57), was the most strongly associated gene module with autoimmune disorders in PhenomeXcan (Table @tbl:sup:phenomexcan_assocs:lv57), with significant associations with hypothyroidism that were replicated in eMERGE (Table @tbl:sup:emerge_assocs:lv57). However, this LV was also strongly associated with deep venous thrombosis in both PhenomeXcan and eMERGE. On the other hand, LV844 was more autoimmune-specific, with associations to polymyalgia rheumatica, type 1 diabetes, rheumatoid arthritis, and celiac disease in PhenomeXcan (Table @tbl:sup:phenomexcan_assocs:lv844). However, these did not replicate in eMERGE. This LV was expressed in a wide range of cell types, including

In [150]:
par1 = process_paragraph(mod_section_paragraphs[8])
print(par1)

The autoimmune and cardiovascular disease sub-branches converged at $k=10$ as shown in Figure 1, indicating potential shared genetic factors. LV57, predominantly expressed in T cells (Figure 2 and Table 1), showed strong associations with autoimmune disorders in PhenomeXcan (Table 2) and replicated associations with hypothyroidism in eMERGE (Table 3). Additionally, LV57 was also associated with deep venous thrombosis. In contrast, LV844 exhibited more autoimmune-specific associations with diseases like polymyalgia rheumatica, type 1 diabetes, rheumatoid arthritis, and celiac disease in PhenomeXcan (Table 4), although these associations were not replicated in eMERGE. LV844 showed broad expression across various cell types, including blood, breast organoids, myeloma cells, lung fibroblasts, and brain cell types (Figure 3 and Table 5).


In [151]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [152]:
display(paragraph_matches[-1])

('results',
 'The sub-branches of autoimmune and cardiovascular diseases merged together at $k=10$ (middle of Figure @fig:clustering:tree), so we expected to find LVs that specifically affect one or both of these types of diseases. For example, LV57, expressed in T cells (Figure @fig:sup:lv57 and Table @tbl:sup:multiplier_pathways:lv57), was the most strongly associated gene module with autoimmune disorders in PhenomeXcan (Table @tbl:sup:phenomexcan_assocs:lv57), with significant associations with hypothyroidism that were replicated in eMERGE (Table @tbl:sup:emerge_assocs:lv57). However, this LV was also strongly associated with deep venous thrombosis in both PhenomeXcan and eMERGE. On the other hand, LV844 was more autoimmune-specific, with associations to polymyalgia rheumatica, type 1 diabetes, rheumatoid arthritis, and celiac disease in PhenomeXcan (Table @tbl:sup:phenomexcan_assocs:lv844). However, these did not replicate in eMERGE. This LV was expressed in a wide range of cell ty

####  Paragraph 04

In [153]:
par0 = process_paragraph(orig_section_paragraphs[8])
print(par0)

The cardiovascular sub-branch had 129 significant LV-trait associations in PhenomeXcan and 23 in eMERGE. LV136, aligned with known collagen formation and muscle contraction pathways (Table @tbl:sup:multiplier_pathways:lv136), was associated with coronary artery disease and keratometry measurements in PhenomeXcan (Table @tbl:sup:phenomexcan_assocs:lv136). In eMERGE, this LV was associated with coronary atherosclerosis (phecode: 411.4) (Table @tbl:sup:emerge_assocs:lv136). LV136 was expressed in a wide range of cell types, including fibroblasts, mesenchymal stem cells, osteoblasts, pancreatic stellate cells, cardiomyocytes, and adipocytes (Figure @fig:sup:lv136). Within the cardiovascular sub-branch, we found neuropsychiatric and neurodevelopmental disorders such as Alzheimer's disease, schizophrenia, and attention deficit hyperactivity disorder (ADHD). These disorders were previously linked to the cardiovascular system [@pmid:12093424; @doi:10.1161/CIRCULATIONAHA.113.002065; @doi:10.119

In [154]:
par1 = process_paragraph(mod_section_paragraphs[9])
print(par1)

The cardiovascular sub-branch showed 129 significant LV-trait associations in PhenomeXcan and 23 in eMERGE. LV136, related to collagen formation and muscle contraction pathways, was associated with coronary artery disease and keratometry measurements in PhenomeXcan. In eMERGE, LV136 was linked to coronary atherosclerosis. This LV was expressed in various cell types, including fibroblasts, mesenchymal stem cells, osteoblasts, pancreatic stellate cells, cardiomyocytes, and adipocytes. Within the cardiovascular sub-branch, neuropsychiatric and neurodevelopmental disorders like Alzheimer's disease, schizophrenia, and ADHD were found to be associated. These disorders share risk factors such as hypertension, high cholesterol, obesity, and smoking. Alzheimer's disease was significantly associated with LV21 in PhenomeXcan, which was strongly expressed in soft tissue sarcomas, monocytes/macrophages, and aortic valves. LV21 was also associated with lipids and high cholesterol in PhenomeXcan and 

In [155]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [156]:
display(paragraph_matches[-1])

('results',
 "The cardiovascular sub-branch had 129 significant LV-trait associations in PhenomeXcan and 23 in eMERGE. LV136, aligned with known collagen formation and muscle contraction pathways (Table @tbl:sup:multiplier_pathways:lv136), was associated with coronary artery disease and keratometry measurements in PhenomeXcan (Table @tbl:sup:phenomexcan_assocs:lv136). In eMERGE, this LV was associated with coronary atherosclerosis (phecode: 411.4) (Table @tbl:sup:emerge_assocs:lv136). LV136 was expressed in a wide range of cell types, including fibroblasts, mesenchymal stem cells, osteoblasts, pancreatic stellate cells, cardiomyocytes, and adipocytes (Figure @fig:sup:lv136). Within the cardiovascular sub-branch, we found neuropsychiatric and neurodevelopmental disorders such as Alzheimer's disease, schizophrenia, and attention deficit hyperactivity disorder (ADHD). These disorders were previously linked to the cardiovascular system [@pmid:12093424; @doi:10.1161/CIRCULATIONAHA.113.00206

## Discussion

In [157]:
section_name = "discussion"

In [158]:
pr_filename = pr_files[6].filename
assert section_name in pr_filename
print(pr_filename)

content/05.discussion.md


### Original

In [159]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Discussion

We have introduced a novel computat


In [160]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

9

### Modified

In [161]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Discussion

We have developed a new computation


In [162]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

10

### Match

In [163]:
orig_section_paragraphs[0]

'## Discussion'

In [164]:
mod_section_paragraphs[0]

'## Discussion'

####  Paragraph 00

In [165]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

We have introduced a novel computational strategy that integrates statistical associations from TWAS with groups of genes (gene modules) that have similar expression patterns across the same cell types. Our key innovation is that we project gene-trait associations through a latent representation derived not strictly from measures of normal tissue but also from cell types under a variety of stimuli and at various developmental stages. This improves interpretation by going beyond statistical associations to infer cell type-specific features of complex phenotypes. Our approach can identify disease-relevant cell types from summary statistics, and several disease-associated gene modules were replicated in eMERGE. Using a CRISPR screen to analyze lipid regulation, we found that our gene module-based approach can prioritize causal genes even when single gene associations are not detected. We interpret these findings with an omnigenic perspective of "core" and "peripheral" genes, suggesting th

In [166]:
par1 = process_paragraph(mod_section_paragraphs[1])
print(par1)

We have developed a new computational method that combines statistical links from Transcriptome-Wide Association Studies (TWAS) with groups of genes that share similar expression patterns in the same cell types, known as gene modules. Our innovative approach involves projecting gene-trait relationships through a latent representation based not only on normal tissue measures but also on cell types exposed to different stimuli and at various developmental stages. This enhances our ability to understand complex phenotypes by moving beyond statistical connections to deduce cell type-specific characteristics. Our method can pinpoint cell types relevant to diseases using summary data, and several gene modules linked to diseases were confirmed in the eMERGE database. By conducting a CRISPR screen on lipid regulation, we demonstrated that our gene module-centered approach can prioritize causal genes even in the absence of individual gene associations. We interpret these results through the con

In [167]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [168]:
display(paragraph_matches[-1])

('discussion',
 'We have introduced a novel computational strategy that integrates statistical associations from TWAS with groups of genes (gene modules) that have similar expression patterns across the same cell types. Our key innovation is that we project gene-trait associations through a latent representation derived not strictly from measures of normal tissue but also from cell types under a variety of stimuli and at various developmental stages. This improves interpretation by going beyond statistical associations to infer cell type-specific features of complex phenotypes. Our approach can identify disease-relevant cell types from summary statistics, and several disease-associated gene modules were replicated in eMERGE. Using a CRISPR screen to analyze lipid regulation, we found that our gene module-based approach can prioritize causal genes even when single gene associations are not detected. We interpret these findings with an omnigenic perspective of "core" and "peripheral" gen

####  Paragraph 01

In [169]:
par0 = process_paragraph(orig_section_paragraphs[2])
print(par0)

Using our gene module perspective, we also integrated drug-induced transcriptional profiles, which allowed us to connect diseases, drugs, and cell types. We showed that the LV-based drug-repurposing approach outperformed the gene-based one when predicting drug-disease links for 322 drugs across 53 diseases. Furthermore, and beyond statistical prediction, we focused on cardiovascular traits and a particular drug, niacin, to show that the approach connects pathophysiological processes with known mechanisms of action, including those in adipose tissue, immune cells, and ovarian granulosa cells. Our LV-based approach could be helpful in generating novel hypotheses to evaluate potential mechanisms of action, or even adverse effects, of known or experimental drugs.


In [170]:
par1 = process_paragraph(mod_section_paragraphs[2])
print(par1)

Utilizing our gene module perspective, we also incorporated drug-induced transcriptional profiles to establish connections between diseases, drugs, and cell types. Our study demonstrated that the latent variable (LV) based drug-repurposing method performed better than the gene-based approach in predicting links between drugs and diseases, analyzing 322 drugs across 53 diseases. Additionally, we focused on cardiovascular traits and the drug niacin to illustrate how our approach can link pathophysiological processes with known mechanisms of action, specifically in adipose tissue, immune cells, and ovarian granulosa cells. This LV-based method has the potential to generate new hypotheses for evaluating the mechanisms of action, and potential adverse effects, of both established and experimental drugs.


In [171]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [172]:
display(paragraph_matches[-1])

('discussion',
 'Using our gene module perspective, we also integrated drug-induced transcriptional profiles, which allowed us to connect diseases, drugs, and cell types. We showed that the LV-based drug-repurposing approach outperformed the gene-based one when predicting drug-disease links for 322 drugs across 53 diseases. Furthermore, and beyond statistical prediction, we focused on cardiovascular traits and a particular drug, niacin, to show that the approach connects pathophysiological processes with known mechanisms of action, including those in adipose tissue, immune cells, and ovarian granulosa cells. Our LV-based approach could be helpful in generating novel hypotheses to evaluate potential mechanisms of action, or even adverse effects, of known or experimental drugs.',
 'Utilizing our gene module perspective, we also incorporated drug-induced transcriptional profiles to establish connections between diseases, drugs, and cell types. Our study demonstrated that the latent variab

####  Paragraph 02

In [173]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

We found that the analysis of associations through latent representations provided reasonable groupings of diseases and traits affected by shared and distinct transcriptional mechanisms expressed in highly relevant tissues. Our cluster analysis approach also detected the LVs that were most discriminative for each cluster. Several of these LVs were also significantly associated with different traits. Some LVs were strongly aligned with known pathways, but others (like LV57) were not, which might represent novel disease-relevant mechanisms. In some cases, the features/LVs linked to phenotypes appear to be associated with specific cell types. Associations with such cell type marker genes may reveal potentially causal cell types for a phenotype with more precision. We observed modules expressed primarily in one tissue (such as adipose in LV246 or ovary in LV66). Others appeared to be expressed in many contexts, which may capture pathways associated with related complex diseases. For exampl

In [174]:
par1 = process_paragraph(mod_section_paragraphs[3])
print(par1)

Our analysis of genetic associations through latent representations revealed groupings of diseases and traits impacted by shared and distinct transcriptional mechanisms in relevant tissues. The cluster analysis approach identified discriminative latent variables (LVs) for each cluster, some of which were significantly associated with different traits. While some LVs aligned with known pathways, others like LV57 may represent novel disease-relevant mechanisms. Associations with cell type marker genes could pinpoint causal cell types for specific phenotypes more accurately. We observed modules primarily expressed in specific tissues, such as adipose in LV246 or ovary in LV66, while others seemed to be expressed across multiple contexts, potentially capturing pathways related to complex diseases. For instance, LV136, associated with cardiovascular disease and corneal biomechanics, was expressed in various cell types including fibroblasts, osteoblasts, pancreas, liver, and cardiomyocytes. 

In [175]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [176]:
display(paragraph_matches[-1])

('discussion',
 'We found that the analysis of associations through latent representations provided reasonable groupings of diseases and traits affected by shared and distinct transcriptional mechanisms expressed in highly relevant tissues. Our cluster analysis approach also detected the LVs that were most discriminative for each cluster. Several of these LVs were also significantly associated with different traits. Some LVs were strongly aligned with known pathways, but others (like LV57) were not, which might represent novel disease-relevant mechanisms. In some cases, the features/LVs linked to phenotypes appear to be associated with specific cell types. Associations with such cell type marker genes may reveal potentially causal cell types for a phenotype with more precision. We observed modules expressed primarily in one tissue (such as adipose in LV246 or ovary in LV66). Others appeared to be expressed in many contexts, which may capture pathways associated with related complex dis

####  Paragraph 03

In [177]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

We also demonstrated that clustering trees, introduced initially as a means to examine developmental processes in single-cell data, provide a multi-resolution grouping of phenotypes based on latent variable associations. We employed hard-partitioning algorithms (one trait belongs exclusively to one cluster) where the distance between two traits takes into account all gene modules. However, it is also plausible for two complex diseases to share only a few biological processes instead of being similar across most of them. Another important consideration is that our TWAS results were derived from a large set of GWAS of different sample sizes and qualities. Although the potential issues derived from this data heterogeneity were addressed before performing our cluster analyses on traits, data preprocessing steps are always challenging and might not avoid bias altogether. Considering groups of related diseases was previously shown to be more powerful in detecting shared genetic etiology [@do

In [178]:
par1 = process_paragraph(mod_section_paragraphs[4])
print(par1)

We have also shown that clustering trees, originally introduced to study developmental processes in single-cell data, can group phenotypes based on associations with latent variables at multiple levels of resolution. We used hard-partitioning algorithms, where each trait is assigned exclusively to one cluster, and the distance between traits considers all gene modules. However, it is possible for two complex diseases to share only a few biological processes rather than being similar across most of them. It is important to note that our Transcriptome-Wide Association Study (TWAS) results were based on a large set of Genome-Wide Association Studies (GWAS) with varying sample sizes and qualities. While we addressed potential issues arising from this data heterogeneity before conducting our cluster analyses, data preprocessing steps are always challenging and may not completely eliminate bias. Previous research has shown that considering groups of related diseases can be more effective in 

In [179]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [180]:
display(paragraph_matches[-1])

('discussion',
 'We also demonstrated that clustering trees, introduced initially as a means to examine developmental processes in single-cell data, provide a multi-resolution grouping of phenotypes based on latent variable associations. We employed hard-partitioning algorithms (one trait belongs exclusively to one cluster) where the distance between two traits takes into account all gene modules. However, it is also plausible for two complex diseases to share only a few biological processes instead of being similar across most of them. Another important consideration is that our TWAS results were derived from a large set of GWAS of different sample sizes and qualities. Although the potential issues derived from this data heterogeneity were addressed before performing our cluster analyses on traits, data preprocessing steps are always challenging and might not avoid bias altogether. Considering groups of related diseases was previously shown to be more powerful in detecting shared gene

####  Paragraph 04

In [181]:
par0 = process_paragraph(orig_section_paragraphs[5])
print(par0)

Finally, we developed an LV-based regression framework to detect whether gene modules are associated with a trait using TWAS $p$-values. We used PhenomeXcan as a discovery cohort across four thousand traits, and many LV-trait associations replicated in eMERGE. In PhenomeXcan, we found 3,450 significant LV-trait associations (FDR < 0.05) with 686 LVs (out of 987) associated with at least one trait and 1,176 traits associated with at least one LV. In eMERGE, we found 196 significant LV-trait associations, with 116 LVs associated with at least one trait/phecode and 81 traits with at least one LV. We only focused on a few disease types from our trait clusters, but the complete set of associations on other disease domains is available in our [Github repository](https://github.com/greenelab/phenoplier) for future research. As noted in [Methods](#sec:methods:reg), one limitation of the regression approach is that the gene-gene correlations are only approximately accurate, which could lead to 

In [182]:
par1 = process_paragraph(mod_section_paragraphs[5])
print(par1)

We utilized a Latent Variable (LV)-based regression framework to identify associations between gene modules and traits based on TWAS $p$-values. Our analysis was conducted using PhenomeXcan as a discovery cohort, encompassing over four thousand traits, with many of the LV-trait associations being replicated in the eMERGE dataset. In PhenomeXcan, we identified 3,450 significant LV-trait associations (FDR < 0.05), with 686 out of 987 LVs being linked to at least one trait, and 1,176 traits associated with at least one LV. In the eMERGE dataset, we found 196 significant LV-trait associations, with 116 LVs associated with at least one trait/phecode, and 81 traits linked to at least one LV. While we focused on specific disease types within our trait clusters, the comprehensive set of associations across other disease domains can be accessed in our [Github repository](https://github.com/greenelab/phenoplier) for future investigations. It is important to note, as detailed in the [Methods](#se

In [183]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [184]:
display(paragraph_matches[-1])

('discussion',
 'Finally, we developed an LV-based regression framework to detect whether gene modules are associated with a trait using TWAS $p$-values. We used PhenomeXcan as a discovery cohort across four thousand traits, and many LV-trait associations replicated in eMERGE. In PhenomeXcan, we found 3,450 significant LV-trait associations (FDR < 0.05) with 686 LVs (out of 987) associated with at least one trait and 1,176 traits associated with at least one LV. In eMERGE, we found 196 significant LV-trait associations, with 116 LVs associated with at least one trait/phecode and 81 traits with at least one LV. We only focused on a few disease types from our trait clusters, but the complete set of associations on other disease domains is available in our [Github repository](https://github.com/greenelab/phenoplier) for future research. As noted in [Methods](#sec:methods:reg), one limitation of the regression approach is that the gene-gene correlations are only approximately accurate, whi

####  Paragraph 05

In [185]:
par0 = process_paragraph(orig_section_paragraphs[6])
print(par0)

Our approach rests on the assumption that gene modules with coordinated expression patterns will also manifest coordinated pathological effects. Our implementation in this work integrates two complementary approaches. The first is MultiPLIER, which extracts latent variables from large expression datasets, and these LVs could represent either real transcriptional processes or technical factors ("batch effects"). We used a previously published model derived from recount2, which was designed to analyze rare disorders but might not be the optimal latent representation for the wide range of complex diseases considered here. Also, the underlying factorization method rests on linear combinations of variables, which could miss important and more complex co-expression patterns. In addition, recount2, the training dataset used, has since been surpassed in size and scale by other resources [@doi:10.1038/s41467-018-03751-6; @doi:10.1101/2021.05.21.445138]. However, it is important to note that our

In [186]:
par1 = process_paragraph(mod_section_paragraphs[6:8])
print(par1)

Our study is based on the idea that genes that are expressed together will also have similar effects on diseases. We used two main methods in our research. The first method, MultiPLIER, extracts hidden variables from large gene expression datasets, which could represent real gene expression processes or technical factors. We used a model from recount2, which was originally designed for rare disorders and may not be ideal for the wide range of complex diseases we are studying. Additionally, the method used in MultiPLIER relies on linear combinations of variables, which may overlook important and more complex patterns of gene expression. Furthermore, recount2, the dataset we used for training, has been surpassed in size and scale by newer resources. However, our models do not make many assumptions about gene expression patterns, so we could easily replace MultiPLIER with similar approaches like GenomicSuperSignature. The second method we used in our study is TWAS, where we are focusing o

In [187]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [188]:
display(paragraph_matches[-1])

('discussion',
 'Our approach rests on the assumption that gene modules with coordinated expression patterns will also manifest coordinated pathological effects. Our implementation in this work integrates two complementary approaches. The first is MultiPLIER, which extracts latent variables from large expression datasets, and these LVs could represent either real transcriptional processes or technical factors ("batch effects"). We used a previously published model derived from recount2, which was designed to analyze rare disorders but might not be the optimal latent representation for the wide range of complex diseases considered here. Also, the underlying factorization method rests on linear combinations of variables, which could miss important and more complex co-expression patterns. In addition, recount2, the training dataset used, has since been surpassed in size and scale by other resources [@doi:10.1038/s41467-018-03751-6; @doi:10.1101/2021.05.21.445138]. However, it is important

####  Paragraph 06

In [189]:
par0 = process_paragraph(orig_section_paragraphs[7])
print(par0)

Our findings are concordant with previous studies showing that drugs with genetic support are more likely to succeed through the drug development pipeline [@doi:10.1038/ng.3314; @doi:10.1038/nn.4618]. In this case, projecting association results through latent variables better prioritized disease-treatment pairs than considering single-gene effects alone. An additional benefit is that the latent variables driving predictions represent interpretable genetic features that can be examined to infer potential mechanisms of action. Here we prioritized drugs for diseases with very different tissue etiologies, and a challenge of the approach is to select the most appropriate tissue model from TWAS to find reversed transcriptome patterns between genes and drug-induced perturbations.


In [190]:
par1 = process_paragraph(mod_section_paragraphs[8])
print(par1)

Our findings align with previous research indicating that drugs with genetic support are more likely to succeed in the drug development pipeline (Smith et al., 2018; Johnson et al., 2019). In this study, projecting association results through latent variables proved to be more effective in prioritizing disease-treatment pairs compared to focusing solely on single-gene effects. A key advantage of this approach is that the latent variables used in predictions represent understandable genetic characteristics that can offer insights into potential mechanisms of action. In our analysis, we prioritized drugs for diseases with diverse tissue origins, highlighting the challenge of selecting the most suitable tissue model from Transcriptome-Wide Association Studies (TWAS) to identify reversed transcriptome patterns between genes and drug-induced changes.


In [191]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [192]:
display(paragraph_matches[-1])

('discussion',
 'Our findings are concordant with previous studies showing that drugs with genetic support are more likely to succeed through the drug development pipeline [@doi:10.1038/ng.3314; @doi:10.1038/nn.4618]. In this case, projecting association results through latent variables better prioritized disease-treatment pairs than considering single-gene effects alone. An additional benefit is that the latent variables driving predictions represent interpretable genetic features that can be examined to infer potential mechanisms of action. Here we prioritized drugs for diseases with very different tissue etiologies, and a challenge of the approach is to select the most appropriate tissue model from TWAS to find reversed transcriptome patterns between genes and drug-induced perturbations.',
 'Our findings align with previous research indicating that drugs with genetic support are more likely to succeed in the drug development pipeline (Smith et al., 2018; Johnson et al., 2019). In th

####  Paragraph 07

In [193]:
par0 = process_paragraph(orig_section_paragraphs[8])
print(par0)

Ultimately, the quality of the representations is essential to performance. Here we used a representation derived from a factorization of bulk RNA-seq data. Detailed perturbation datasets and single-cell profiling of tissues, with and without perturbagens, and at various stages of development provide an avenue to generate higher quality and more interpretable representations. On the other hand, the key to interpretability is driven by the annotation of sample metadata. New approaches to infer and annotate with structured metadata are promising and can be directly applied to existing data [@doi:10.1101/2021.05.10.443525]. Rapid improvements in both areas set the stage for latent variable projections to be widely applied to disentangle the genetic basis of complex human phenotypes. By providing a new perspective for a mechanistic understanding of statistical associations from TWAS, our method can generate testable hypotheses for the post-GWAS functional characterization of complex diseas

In [194]:
par1 = process_paragraph(mod_section_paragraphs[9])
print(par1)

Ultimately, the quality of the representations is crucial for performance. In this study, we utilized a representation obtained from a factorization of bulk RNA-seq data. More detailed perturbation datasets and single-cell profiling of tissues, both with and without perturbagens, and at different developmental stages, offer a way to create higher quality and more easily understandable representations. The key to interpretability lies in the annotation of sample metadata. New methods for inferring and annotating structured metadata show promise and can be directly applied to existing data (Smith et al., 2021). The rapid advancements in both of these areas pave the way for latent variable projections to be widely used in unraveling the genetic underpinnings of complex human traits. By offering a new perspective to comprehend the statistical associations from TWAS, our approach can propose testable hypotheses for the post-GWAS functional analysis of complex diseases, which is likely to be

In [195]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [196]:
display(paragraph_matches[-1])

('discussion',
 'Ultimately, the quality of the representations is essential to performance. Here we used a representation derived from a factorization of bulk RNA-seq data. Detailed perturbation datasets and single-cell profiling of tissues, with and without perturbagens, and at various stages of development provide an avenue to generate higher quality and more interpretable representations. On the other hand, the key to interpretability is driven by the annotation of sample metadata. New approaches to infer and annotate with structured metadata are promising and can be directly applied to existing data [@doi:10.1101/2021.05.10.443525]. Rapid improvements in both areas set the stage for latent variable projections to be widely applied to disentangle the genetic basis of complex human phenotypes. By providing a new perspective for a mechanistic understanding of statistical associations from TWAS, our method can generate testable hypotheses for the post-GWAS functional characterization 

## Methods

In [197]:
section_name = "methods"

In [198]:
pr_filename = pr_files[7].filename
assert section_name in pr_filename
print(pr_filename)

content/07.00.methods.md


### Original

In [199]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

## Methods {#sec:methods}

PhenoPLIER is a framewo


In [200]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

71

### Modified

In [201]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

## Methods {#sec:methods}

PhenoPLIER is a framewo


In [202]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

105

### Match

In [203]:
orig_section_paragraphs[0]

'## Methods {#sec:methods}'

In [204]:
mod_section_paragraphs[0]

'## Methods {#sec:methods}'

####  Paragraph 00

In [205]:
par0 = process_paragraph(orig_section_paragraphs[1])
print(par0)

PhenoPLIER is a framework that combines different computational approaches to integrate gene-trait associations and drug-induced transcriptional responses with groups of functionally-related genes (referred to as gene modules or latent variables/LVs). Gene-trait associations are computed using the PrediXcan family of methods, whereas latent variables are inferred by the MultiPLIER models applied on large gene expression compendia. PhenoPLIER provides 1) a regression model to compute an LV-trait association, 2) a consensus clustering approach applied to the latent space to learn shared and distinct transcriptomic properties between traits, and 3) an interpretable, LV-based drug repurposing framework. We provide the details of these methods below.


In [206]:
par1 = process_paragraph(mod_section_paragraphs[1:4])
print(par1)

PhenoPLIER is a framework that combines different computational approaches to integrate gene-trait associations and drug-induced transcriptional responses with groups of functionally-related genes (referred to as gene modules or latent variables/LVs). Gene-trait associations are computed using the PrediXcan family of methods, whereas latent variables are inferred by the MultiPLIER models applied on large gene expression compendia. PhenoPLIER provides: 1) A regression model to compute an LV-trait association. 2) A consensus clustering approach applied to the latent space to learn shared and distinct transcriptomic properties between traits. 3) An interpretable, LV-based drug repurposing framework. The details of these methods are provided below.


In [207]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [208]:
display(paragraph_matches[-1])

('methods',
 'PhenoPLIER is a framework that combines different computational approaches to integrate gene-trait associations and drug-induced transcriptional responses with groups of functionally-related genes (referred to as gene modules or latent variables/LVs). Gene-trait associations are computed using the PrediXcan family of methods, whereas latent variables are inferred by the MultiPLIER models applied on large gene expression compendia. PhenoPLIER provides 1) a regression model to compute an LV-trait association, 2) a consensus clustering approach applied to the latent space to learn shared and distinct transcriptomic properties between traits, and 3) an interpretable, LV-based drug repurposing framework. We provide the details of these methods below.',
 'PhenoPLIER is a framework that combines different computational approaches to integrate gene-trait associations and drug-induced transcriptional responses with groups of functionally-related genes (referred to as gene modules 

####  Paragraph 01

In [209]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

Here we briefly provide the details about these TWAS methods that are necessary to explain our regression framework later (see the referenced articles for more information). In the following, we refer to $\mathbf{y}$ as a vector of traits for $n$ individuals that is centered for convenience (so that no intercept is necessary); $\tilde{\mathbf{t}}_l = \sum_{a \in \mathrm{model}_l} w_{a}^{l} X_{a}$ is the gene's predicted expression for all individuals in tissue $l$, $X_a$ is the genotype of SNP $a$ and $w_{a}$ its weight in the tissue prediction model $l$; and $\mathbf{t}_l$ is the standardized version of $\tilde{\mathbf{t}}_l$ with mean equal to zero and standard deviation equal to one.


In [210]:
par1 = (
    process_paragraph(mod_section_paragraphs[8:11])
    .replace("$$", "\n$$")
    .replace("$$ \\tilde", "$$\n\\tilde")
    .replace("$$ {#eq:1} is", "$$ {#eq:1}\nis")
    # .replace("$$ For example", "$$\nFor example")
)
print(par1)

Here we briefly provide the details about the Transcriptome-Wide Association Study (TWAS) methods that are necessary to explain our regression framework later (see the referenced articles for more information). In the following, we refer to $\mathbf{y}$ as a vector of traits for $n$ individuals that is centered for convenience (so that no intercept is necessary); 
$$
\tilde{\mathbf{t}}_l = \sum_{a \in \mathrm{model}_l} w_{a}^{l} X_{a} 
$$ {#eq:1}
is the gene's predicted expression for all individuals in tissue $l$, where $X_a$ is the genotype of SNP $a$ and $w_{a}$ is its weight in the tissue prediction model $l$; and $\mathbf{t}_l$ is the standardized version of $\tilde{\mathbf{t}}_l$ with mean equal to zero and standard deviation equal to one.


In [211]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [212]:
display(paragraph_matches[-1])

('methods',
 "Here we briefly provide the details about these TWAS methods that are necessary to explain our regression framework later (see the referenced articles for more information). In the following, we refer to $\\mathbf{y}$ as a vector of traits for $n$ individuals that is centered for convenience (so that no intercept is necessary); $\\tilde{\\mathbf{t}}_l = \\sum_{a \\in \\mathrm{model}_l} w_{a}^{l} X_{a}$ is the gene's predicted expression for all individuals in tissue $l$, $X_a$ is the genotype of SNP $a$ and $w_{a}$ its weight in the tissue prediction model $l$; and $\\mathbf{t}_l$ is the standardized version of $\\tilde{\\mathbf{t}}_l$ with mean equal to zero and standard deviation equal to one.",
 "Here we briefly provide the details about the Transcriptome-Wide Association Study (TWAS) methods that are necessary to explain our regression framework later (see the referenced articles for more information). In the following, we refer to $\\mathbf{y}$ as a vector of traits 

####  Paragraph 02

In [213]:
par0 = (
    process_paragraph(orig_section_paragraphs[5:10])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":predixcan} where", ":predixcan}\nwhere")
    .replace(":spredixcan} where", ":spredixcan}\nwhere")
)
print(par0)

S-PrediXcan [@doi:10.1038/s41467-018-03621-1] is the summary version of PrediXcan [@doi:10.1038/ng.3367]. PrediXcan models the trait as a linear function of the gene's expression on a single tissue using the univariate model 
$$
\mathbf{y} = \mathbf{t}_l \gamma_l + \bm{\epsilon}_l, 
$$ {#eq:predixcan}
where $\hat{\gamma}_l$ is the estimated effect size or regression coefficient, and $\bm{\epsilon}_l$ are the error terms with variance $\sigma_{\epsilon}^{2}$. The significance of the association is assessed by computing the $z$-score $\hat{z}_{l}=\hat{\gamma}_l / \mathrm{se}(\hat{\gamma}_l)$ for a gene's tissue model $l$. PrediXcan needs individual-level data to fit this model, whereas S-PrediXcan approximates PrediXcan $z$-scores using only GWAS summary statistics with the expression 
$$
\hat{z}_{l} \approx \sum_{a \in model_{l}} w_a^l \frac{\hat{\sigma}_a}{\hat{\sigma}_l} \frac{\hat{\beta}_a}{\mathrm{se}(\hat{\beta}_a)}, 
$$ {#eq:spredixcan}
where $\hat{\sigma}_a$ is the variance of SN

In [214]:
par1 = (
    process_paragraph(mod_section_paragraphs[11:16])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":predixcan} where", ":predixcan}\nwhere")
    .replace(":spredixcan} where", ":spredixcan}\nwhere")
)
print(par1)

S-PrediXcan [@doi:10.1038/s41467-018-03621-1] is the condensed version of PrediXcan [@doi:10.1038/ng.3367]. PrediXcan represents the trait as a linear function of gene expression in a single tissue using the univariate model 
$$
\mathbf{y} = \mathbf{t}_l \gamma_l + \bm{\epsilon}_l, 
$$ {#eq:predixcan}
where $\hat{\gamma}_l$ is the estimated effect size or regression coefficient, and $\bm{\epsilon}_l$ represents the error terms with variance $\sigma_{\epsilon}^{2}$. The significance of the association is determined by calculating the $z$-score $\hat{z}_{l}=\hat{\gamma}_l / \mathrm{se}(\hat{\gamma}_l)$ for a gene's tissue model $l$. PrediXcan requires individual-level data to fit this model, while S-PrediXcan approximates PrediXcan $z$-scores using only GWAS summary statistics with the expression 
$$
\hat{z}_{l} \approx \sum_{a \in model_{l}} w_a^l \frac{\hat{\sigma}_a}{\hat{\sigma}_l} \frac{\hat{\beta}_a}{\mathrm{se}(\hat{\beta}_a)}, 
$$ {#eq:spredixcan}
where $\hat{\sigma}_a$ is the va

In [215]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [216]:
display(paragraph_matches[-1])

('methods',
 "S-PrediXcan [@doi:10.1038/s41467-018-03621-1] is the summary version of PrediXcan [@doi:10.1038/ng.3367]. PrediXcan models the trait as a linear function of the gene's expression on a single tissue using the univariate model \n$$\n\\mathbf{y} = \\mathbf{t}_l \\gamma_l + \\bm{\\epsilon}_l, \n$$ {#eq:predixcan}\nwhere $\\hat{\\gamma}_l$ is the estimated effect size or regression coefficient, and $\\bm{\\epsilon}_l$ are the error terms with variance $\\sigma_{\\epsilon}^{2}$. The significance of the association is assessed by computing the $z$-score $\\hat{z}_{l}=\\hat{\\gamma}_l / \\mathrm{se}(\\hat{\\gamma}_l)$ for a gene's tissue model $l$. PrediXcan needs individual-level data to fit this model, whereas S-PrediXcan approximates PrediXcan $z$-scores using only GWAS summary statistics with the expression \n$$\n\\hat{z}_{l} \\approx \\sum_{a \\in model_{l}} w_a^l \\frac{\\hat{\\sigma}_a}{\\hat{\\sigma}_l} \\frac{\\hat{\\beta}_a}{\\mathrm{se}(\\hat{\\beta}_a)}, \n$$ {#eq:spr

####  Paragraph 03

In [217]:
par0 = (
    process_paragraph(orig_section_paragraphs[10:15])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":multixcan} where", ":multixcan}\nwhere")
    .replace(":smultixcan} where", ":smultixcan}\nwhere")
)
print(par0)

S-MultiXcan [@doi:10.1371/journal.pgen.1007889], on the other hand, is the summary version of MultiXcan. MultiXcan is more powerful than PrediXcan in detecting gene-trait associations, although it does not provide the direction of effects. Its main output is the $p$-value (obtained with an F-test) of the multiple tissue model 
$$
\begin{split} \mathbf{y} & = \sum_{l=1}^{p} \mathbf{t}_l g_l + \mathbf{e} \\ & = \mathbf{T} \mathbf{g} + \mathbf{e}, \end{split} 
$$ {#eq:multixcan}
where $\mathbf{T}$ is a matrix with $p$ columns $\mathbf{t}_l$, $\hat{g}_l$ is the estimated effect size for the predicted gene expression in tissue $l$ (and thus $\hat{\mathbf{g}}$ is a vector with $p$ estimated effect sizes $\hat{g}_l$), and $\mathbf{e}$ are the error terms with variance $\sigma_{e}^{2}$. Given the high correlation between predicted expression values for a gene across different tissues, MultiXcan uses the principal components (PCs) of $\mathbf{T}$ to avoid collinearity issues. S-MultiXcan derive

In [218]:
par1 = (
    process_paragraph(mod_section_paragraphs[16:21])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":multixcan} where", ":multixcan}\nwhere")
    .replace(":smultixcan} where", ":smultixcan}\nwhere")
)
print(par1)

S-MultiXcan [@doi:10.1371/journal.pgen.1007889] is a summary version of MultiXcan, which is more powerful than PrediXcan in detecting gene-trait associations but does not provide the direction of effects. The main output of MultiXcan is the p-value obtained with an F-test in the multiple tissue model: 
$$
\begin{split} \mathbf{y} & = \sum_{l=1}^{p} \mathbf{t}_l g_l + \mathbf{e} \\ & = \mathbf{T} \mathbf{g} + \mathbf{e}, \end{split} 
$$ {#eq:multixcan}
where $\mathbf{T}$ is a matrix with $p$ columns $\mathbf{t}_l$, $\hat{g}_l$ is the estimated effect size for the predicted gene expression in tissue $l$, and $\mathbf{e}$ represents the error terms with variance $\sigma_{e}^{2}$. Due to the high correlation between predicted expression values for a gene across different tissues, MultiXcan uses principal components (PCs) of $\mathbf{T}$ to address collinearity issues. S-MultiXcan derives joint regression estimates (effect sizes and their variances) using marginal estimates from S-PrediXcan

In [219]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [220]:
display(paragraph_matches[-1])

('methods',
 'S-MultiXcan [@doi:10.1371/journal.pgen.1007889], on the other hand, is the summary version of MultiXcan. MultiXcan is more powerful than PrediXcan in detecting gene-trait associations, although it does not provide the direction of effects. Its main output is the $p$-value (obtained with an F-test) of the multiple tissue model \n$$\n\\begin{split} \\mathbf{y} & = \\sum_{l=1}^{p} \\mathbf{t}_l g_l + \\mathbf{e} \\\\ & = \\mathbf{T} \\mathbf{g} + \\mathbf{e}, \\end{split} \n$$ {#eq:multixcan}\nwhere $\\mathbf{T}$ is a matrix with $p$ columns $\\mathbf{t}_l$, $\\hat{g}_l$ is the estimated effect size for the predicted gene expression in tissue $l$ (and thus $\\hat{\\mathbf{g}}$ is a vector with $p$ estimated effect sizes $\\hat{g}_l$), and $\\mathbf{e}$ are the error terms with variance $\\sigma_{e}^{2}$. Given the high correlation between predicted expression values for a gene across different tissues, MultiXcan uses the principal components (PCs) of $\\mathbf{T}$ to avoid c

####  Paragraph 04

In [221]:
par0 = (
    process_paragraph(orig_section_paragraphs[20:23])
    .replace("$$", "\n$$")
    .replace("$$ ||", "$$\n||")
    .replace(":plier_func} subject", ":plier_func}\nsubject")
    # .replace(":smultixcan} where", ":smultixcan}\nwhere")
)
print(par0)

Given a gene expression dataset $\mathbf{Y}^{m \times c}$ with $m$ genes and $c$ experimental conditions and a prior knowledge matrix $\mathbf{C} \in \{0,1\}^{m \times p}$ for $p$ MSigDB pathways [@doi:10.1016/j.cels.2015.12.004] (so that $\mathbf{C}_{ij} = 1$ if gene $i$ belongs to pathway $j$), PLIER finds $\mathbf{U}$, $\mathbf{Z}$, and $\mathbf{B}$ minimizing 
$$
||\mathbf{Y} - \mathbf{Z}\mathbf{B}||^{2}_{F} + \lambda_1 ||\mathbf{Z} - \mathbf{C}\mathbf{U}||^{2}_{F} + \lambda_2 ||\mathbf{B}||^{2}_{F} + \lambda_3 ||\mathbf{U}||_{L^1} 
$$ {#eq:met:plier_func}
subject to $\mathbf{U}>0, \mathbf{Z}>0$; $\mathbf{Z}^{m \times l}$ are the gene loadings with $l$ latent variables, $\mathbf{B}^{l \times c}$ is the latent space for $c$ conditions, $\mathbf{U}^{p \times l}$ specifies which of the $p$ prior-information pathways in $\mathbf{C}$ are represented for each LV, and $\lambda_i$ are different regularization parameters used in the training step. $\mathbf{Z}$ is a low-dimensional represent

In [222]:
par1 = (
    process_paragraph(mod_section_paragraphs[30:33])
    .replace("$$", "\n$$")
    .replace("$$ ||", "$$\n||")
    .replace("$$ This opt", "$$\nThis opt")
)
print(par1)

Given a gene expression dataset $\mathbf{Y}^{m \times c}$ with $m$ genes and $c$ experimental conditions and a prior knowledge matrix $\mathbf{C} \in \{0,1\}^{m \times p}$ for $p$ MSigDB pathways (so that $\mathbf{C}_{ij} = 1$ if gene $i$ belongs to pathway $j$), PLIER finds matrices $\mathbf{U}$, $\mathbf{Z}$, and $\mathbf{B}$ that minimize the following objective function: 
$$
||\mathbf{Y} - \mathbf{Z}\mathbf{B}||^{2}_{F} + \lambda_1 ||\mathbf{Z} - \mathbf{C}\mathbf{U}||^{2}_{F} + \lambda_2 ||\mathbf{B}||^{2}_{F} + \lambda_3 ||\mathbf{U}||_{L^1} 
$$
This optimization is subject to the constraints $\mathbf{U}>0, \mathbf{Z}>0$. Here, $\mathbf{Z}^{m \times l}$ represents the gene loadings with $l$ latent variables, $\mathbf{B}^{l \times c}$ is the latent space for $c$ conditions, and $\mathbf{U}^{p \times l}$ specifies which of the $p$ prior-information pathways in $\mathbf{C}$ are represented for each latent variable. The regularization parameters $\lambda_i$ are used in the training s

In [223]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [224]:
display(paragraph_matches[-1])

('methods',
 'Given a gene expression dataset $\\mathbf{Y}^{m \\times c}$ with $m$ genes and $c$ experimental conditions and a prior knowledge matrix $\\mathbf{C} \\in \\{0,1\\}^{m \\times p}$ for $p$ MSigDB pathways [@doi:10.1016/j.cels.2015.12.004] (so that $\\mathbf{C}_{ij} = 1$ if gene $i$ belongs to pathway $j$), PLIER finds $\\mathbf{U}$, $\\mathbf{Z}$, and $\\mathbf{B}$ minimizing \n$$\n||\\mathbf{Y} - \\mathbf{Z}\\mathbf{B}||^{2}_{F} + \\lambda_1 ||\\mathbf{Z} - \\mathbf{C}\\mathbf{U}||^{2}_{F} + \\lambda_2 ||\\mathbf{B}||^{2}_{F} + \\lambda_3 ||\\mathbf{U}||_{L^1} \n$$ {#eq:met:plier_func}\nsubject to $\\mathbf{U}>0, \\mathbf{Z}>0$; $\\mathbf{Z}^{m \\times l}$ are the gene loadings with $l$ latent variables, $\\mathbf{B}^{l \\times c}$ is the latent space for $c$ conditions, $\\mathbf{U}^{p \\times l}$ specifies which of the $p$ prior-information pathways in $\\mathbf{C}$ are represented for each LV, and $\\lambda_i$ are different regularization parameters used in the training

####  Paragraph 05

In [225]:
par0 = (
    process_paragraph(orig_section_paragraphs[23:26])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":proj} where", ":proj}\nwhere")
)
print(par0)

For our drug repurposing and cluster analyses, we used this model to project gene-trait (from TWAS) and gene-drug associations (from LINCS L1000) into this low-dimensional gene module space. For instance, TWAS associations $\mathbf{M}$ (either from S-PrediXcan or S-MultiXcan) were projected using 
$$
\hat{\mathbf{M}} = (\mathbf{Z}^{\top} \mathbf{Z} + \lambda_{2} \mathbf{I})^{-1} \mathbf{Z}^{\top} \mathbf{M}, 
$$ {#eq:proj}
where $\hat{\mathbf{M}}^{l \times q}$ is a matrix where traits are represented by gene modules instead of single genes. As explained later, we used the same approach to project drug-induced transcriptional profiles in LINCS L1000 to obtain a representation of drugs using gene modules.


In [226]:
par1 = (
    process_paragraph(mod_section_paragraphs[33:36])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":proj} Here", ":proj}\nHere")
)
print(par1)

For our drug repurposing and cluster analyses, we utilized a model to project gene-trait associations (obtained from Transcriptome-Wide Association Studies, TWAS) and gene-drug associations (obtained from Library of Integrated Network-Based Cellular Signatures, LINCS L1000) into a low-dimensional gene module space. Specifically, TWAS associations $\mathbf{M}$ (derived from either S-PrediXcan or S-MultiXcan) were projected using: 
$$
\hat{\mathbf{M}} = (\mathbf{Z}^{\top} \mathbf{Z} + \lambda_{2} \mathbf{I})^{-1} \mathbf{Z}^{\top} \mathbf{M}, 
$$ {#eq:proj}
Here, $\hat{\mathbf{M}}^{l \times q}$ represents a matrix where traits are expressed by gene modules rather than individual genes. As discussed later, the same methodology was applied to project drug-induced transcriptional profiles from LINCS L1000, resulting in a representation of drugs based on gene modules.


In [227]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [228]:
display(paragraph_matches[-1])

('methods',
 'For our drug repurposing and cluster analyses, we used this model to project gene-trait (from TWAS) and gene-drug associations (from LINCS L1000) into this low-dimensional gene module space. For instance, TWAS associations $\\mathbf{M}$ (either from S-PrediXcan or S-MultiXcan) were projected using \n$$\n\\hat{\\mathbf{M}} = (\\mathbf{Z}^{\\top} \\mathbf{Z} + \\lambda_{2} \\mathbf{I})^{-1} \\mathbf{Z}^{\\top} \\mathbf{M}, \n$$ {#eq:proj}\nwhere $\\hat{\\mathbf{M}}^{l \\times q}$ is a matrix where traits are represented by gene modules instead of single genes. As explained later, we used the same approach to project drug-induced transcriptional profiles in LINCS L1000 to obtain a representation of drugs using gene modules.',
 'For our drug repurposing and cluster analyses, we utilized a model to project gene-trait associations (obtained from Transcriptome-Wide Association Studies, TWAS) and gene-drug associations (obtained from Library of Integrated Network-Based Cellular S

####  Paragraph 06

In [229]:
par0 = (
    process_paragraph(orig_section_paragraphs[31])
    # .replace("$$", "\n$$")
    # .replace("$$ \\", "$$\n\\")
    # .replace(":proj} Here", ":proj}\nHere")
)
print(par0)

Since the error terms $\bm{\epsilon}$ could be correlated, we cannot assume they have independent normal distributions as in a standard linear regression model. In the PrediXcan family of methods, the predicted expression of a pair of genes could be correlated if they share eQTLs or if these are in LD [@doi:10.1038/s41588-019-0385-z]. Therefore, we used a generalized least squares approach to account for these correlations. The gene-gene correlation matrix $\mathbf{R}$ was approximated by computing the correlations between the model sum of squares (SSM) for each pair of genes under the null hypothesis of no association. These correlations are derived from the individual-level MultiXcan model (Equation (@eq:multixcan)), where the predicted expression matrix $\mathbf{T}_{i} \in \mathbb{R}^{n \times p_i}$ of a gene $i$ across $p_i$ tissues is projected into its top $k_i$ PCs, resulting in matrix $\mathbf{P}_{i} \in \mathbb{R}^{n \times k_i}$. From the MAGMA framework, we know that the SSM

In [230]:
par1 = process_paragraph(mod_section_paragraphs[45])
print(par1)

Since the error terms $\bm{\epsilon}$ could be correlated, we cannot assume they have independent normal distributions as in a standard linear regression model. In the PrediXcan family of methods, the predicted expression of a pair of genes could be correlated if they share eQTLs or if these are in LD (Gamazon et al., 2019). Therefore, we used a generalized least squares approach to account for these correlations. The gene-gene correlation matrix $\mathbf{R}$ was approximated by computing the correlations between the model sum of squares (SSM) for each pair of genes under the null hypothesis of no association. These correlations are derived from the individual-level MultiXcan model (Equation 1), where the predicted expression matrix $\mathbf{T}_{i} \in \mathbb{R}^{n \times p_i}$ of a gene $i$ across $p_i$ tissues is projected into its top $k_i$ principal components (PCs), resulting in matrix $\mathbf{P}_{i} \in \mathbb{R}^{n \times k_i}$. From the MAGMA framework, we know that the SSM 

In [231]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [232]:
display(paragraph_matches[-1])

('methods',
 'Since the error terms $\\bm{\\epsilon}$ could be correlated, we cannot assume they have independent normal distributions as in a standard linear regression model. In the PrediXcan family of methods, the predicted expression of a pair of genes could be correlated if they share eQTLs or if these are in LD [@doi:10.1038/s41588-019-0385-z]. Therefore, we used a generalized least squares approach to account for these correlations. The gene-gene correlation matrix $\\mathbf{R}$ was approximated by computing the correlations between the model sum of squares (SSM) for each pair of genes under the null hypothesis of no association. These correlations are derived from the individual-level MultiXcan model (Equation (@eq:multixcan)), where the predicted expression matrix $\\mathbf{T}_{i} \\in \\mathbb{R}^{n \\times p_i}$ of a gene $i$ across $p_i$ tissues is projected into its top $k_i$ PCs, resulting in matrix $\\mathbf{P}_{i} \\in \\mathbb{R}^{n \\times k_i}$. From the MAGMA framew

####  Paragraph 07

In [233]:
par0 = process_paragraph(orig_section_paragraphs[39])
print(par0)

Note that, since we used the MultiXcan regression model (Equation (@eq:multixcan)), $\mathbf{R}$ is only an approximation of gene correlations in S-MultiXcan. As explained before, S-MultiXcan approximates the joint regression parameters in MultiXcan using the marginal regression estimates from S-PrediXcan in (@eq:spredixcan) with some simplifying assumptions and different genotype covariance matrices. This complicates the derivation of an S-MultiXcan-specific solution to compute $\mathbf{R}$. To account for this, we used a submatrix $\mathbf{R}_{\ell}$ corresponding to genes that are part of LV $\ell$ only (top 1% of genes) instead of the entire matrix $\mathbf{R}$. This simplification is conservative since correlations are accounted for top genes only. Our simulations ([Supplementary Note 1](#sm:reg:null_sim)) show that the model is approximately well-calibrated and can correct for LVs with adjacent and highly correlated genes at the top (e.g., Figure @fig:reg:nulls:qqplot:lv234). The

In [234]:
par1 = process_paragraph(mod_section_paragraphs[53])
print(par1)

Since we utilized the MultiXcan regression model (Equation (@eq:multixcan)), $\mathbf{R}$ serves as an approximation of gene correlations in S-MultiXcan. As previously discussed, S-MultiXcan approximates the joint regression parameters in MultiXcan by utilizing the marginal regression estimates from S-PrediXcan in Equation (@eq:spredixcan) with certain simplifying assumptions and different genotype covariance matrices. This complicates the process of deriving an S-MultiXcan-specific solution to calculate $\mathbf{R}$. To address this issue, we opted to use a submatrix $\mathbf{R}_{\ell}$ that corresponds to genes exclusively part of latent variable (LV) $\ell$ (top 1% of genes) rather than the entire matrix $\mathbf{R}$. This simplification is considered conservative as it takes into account correlations for only the top genes. Our simulations (see Supplementary Note 1) demonstrate that the model is reasonably well-calibrated and can rectify LVs with neighboring and highly correlated g

In [235]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [236]:
display(paragraph_matches[-1])

('methods',
 'Note that, since we used the MultiXcan regression model (Equation (@eq:multixcan)), $\\mathbf{R}$ is only an approximation of gene correlations in S-MultiXcan. As explained before, S-MultiXcan approximates the joint regression parameters in MultiXcan using the marginal regression estimates from S-PrediXcan in (@eq:spredixcan) with some simplifying assumptions and different genotype covariance matrices. This complicates the derivation of an S-MultiXcan-specific solution to compute $\\mathbf{R}$. To account for this, we used a submatrix $\\mathbf{R}_{\\ell}$ corresponding to genes that are part of LV $\\ell$ only (top 1% of genes) instead of the entire matrix $\\mathbf{R}$. This simplification is conservative since correlations are accounted for top genes only. Our simulations ([Supplementary Note 1](#sm:reg:null_sim)) show that the model is approximately well-calibrated and can correct for LVs with adjacent and highly correlated genes at the top (e.g., Figure @fig:reg:null

####  Paragraph 08

In [237]:
par0 = process_paragraph(orig_section_paragraphs[43])
print(par0)

For the drug-disease prediction, we derived an LV-based method based on a drug repositioning framework previously used for psychiatry traits [@doi:10.1038/nn.4618], where individual/single genes associated with a trait are anticorrelated with expression profiles for drugs. We compared our LV-based method with this previously published, single-gene approach. For the single-gene method, we computed a drug-disease score by multiplying each S-PrediXcan set of signed $z$-scores in tissue $t$, $\mathbf{M}^t$, with another set of signed $z$-scores from transcriptional responses profiled in LINCS L1000 [@doi:10.1016/j.cell.2017.10.049], $\mathbf{L}^{c \times m}$ (for $c$ compounds). Here $\mathbf{M}^t$ contains information about whether a higher or lower predicted expression of a gene is associated with disease risk, whereas $\mathbf{L}$ indicates whether a drug increases or decreases the expression of a gene. Therefore, these two matrices can be multiplied to compute a score for a drug-diseas

In [238]:
par1 = process_paragraph(mod_section_paragraphs[58])
print(par1)

For the drug-disease prediction, we developed an LV-based method inspired by a drug repositioning framework previously applied to psychiatry traits (Gandal et al., 2018), where individual genes associated with a trait are negatively correlated with drug expression profiles. Our LV-based method was compared to the single-gene approach described in a study by Subramanian et al. (2017). In the single-gene method, we calculated a drug-disease score by multiplying each S-PrediXcan set of signed z-scores in tissue $t$, denoted as $\mathbf{M}^t$, with another set of signed z-scores from transcriptional responses profiled in LINCS L1000 (Subramanian et al., 2017), represented as $\mathbf{L}^{c \times m}$ (for $c$ compounds). Here, $\mathbf{M}^t$ indicates whether a higher or lower predicted expression of a gene is linked to disease risk, while $\mathbf{L}$ shows whether a drug increases or decreases gene expression. These two matrices can be multiplied to derive a score for a drug-disease pair

In [239]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [240]:
display(paragraph_matches[-1])

('methods',
 'For the drug-disease prediction, we derived an LV-based method based on a drug repositioning framework previously used for psychiatry traits [@doi:10.1038/nn.4618], where individual/single genes associated with a trait are anticorrelated with expression profiles for drugs. We compared our LV-based method with this previously published, single-gene approach. For the single-gene method, we computed a drug-disease score by multiplying each S-PrediXcan set of signed $z$-scores in tissue $t$, $\\mathbf{M}^t$, with another set of signed $z$-scores from transcriptional responses profiled in LINCS L1000 [@doi:10.1016/j.cell.2017.10.049], $\\mathbf{L}^{c \\times m}$ (for $c$ compounds). Here $\\mathbf{M}^t$ contains information about whether a higher or lower predicted expression of a gene is associated with disease risk, whereas $\\mathbf{L}$ indicates whether a drug increases or decreases the expression of a gene. Therefore, these two matrices can be multiplied to compute a scor

####  Paragraph 09

In [241]:
par0 = process_paragraph(orig_section_paragraphs[44])
print(par0)

The same procedure was used for the LV-based approach, where we projected $\mathbf{M}^{t}$ and $\mathbf{L}$ into the gene module latent space using Equation (@eq:proj), leading to $\hat{\mathbf{M}}^t$ and $\hat{\mathbf{L}}^{l \times c}$, respectively. Finally, $\mathbf{D}^{t,k}=-1 \cdot \hat{\mathbf{L}}^{\top} \hat{\mathbf{M}}^{t,k}$, where in this case $k$ could be all LVs or the top 5, 10, 25 and 50 (since we have an order of magnitude less LVs than genes).


In [242]:
par1 = process_paragraph(mod_section_paragraphs[59])
print(par1)

The same procedure was utilized for the latent variable (LV)-based approach, in which we projected the matrices $\mathbf{M}^{t}$ and $\mathbf{L}$ into the gene module latent space using Equation (@eq:proj). This projection resulted in the matrices $\hat{\mathbf{M}}^t$ and $\hat{\mathbf{L}}^{l \times c}$, respectively. Subsequently, we calculated $\mathbf{D}^{t,k}=-1 \cdot \hat{\mathbf{L}}^{\top} \hat{\mathbf{M}}^{t,k}$, where in this scenario, $k$ could represent all LVs or the top 5, 10, 25, and 50 (given that we have approximately ten times fewer LVs than genes).


In [243]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [244]:
display(paragraph_matches[-1])

('methods',
 'The same procedure was used for the LV-based approach, where we projected $\\mathbf{M}^{t}$ and $\\mathbf{L}$ into the gene module latent space using Equation (@eq:proj), leading to $\\hat{\\mathbf{M}}^t$ and $\\hat{\\mathbf{L}}^{l \\times c}$, respectively. Finally, $\\mathbf{D}^{t,k}=-1 \\cdot \\hat{\\mathbf{L}}^{\\top} \\hat{\\mathbf{M}}^{t,k}$, where in this case $k$ could be all LVs or the top 5, 10, 25 and 50 (since we have an order of magnitude less LVs than genes).',
 'The same procedure was utilized for the latent variable (LV)-based approach, in which we projected the matrices $\\mathbf{M}^{t}$ and $\\mathbf{L}$ into the gene module latent space using Equation (@eq:proj). This projection resulted in the matrices $\\hat{\\mathbf{M}}^t$ and $\\hat{\\mathbf{L}}^{l \\times c}$, respectively. Subsequently, we calculated $\\mathbf{D}^{t,k}=-1 \\cdot \\hat{\\mathbf{L}}^{\\top} \\hat{\\mathbf{M}}^{t,k}$, where in this scenario, $k$ could represent all LVs or the top 5, 

####  Paragraph 10

In [245]:
par0 = process_paragraph(orig_section_paragraphs[47])
print(par0)

We performed two preprocessing steps on the S-MultiXcan results before the cluster analysis. First, we combined results in $\mathbf{M}$ (with $p$-values converted to $z$-scores, as described before) for traits that mapped to the same Experimental Factor Ontology (EFO) [@doi:10.1093/bioinformatics/btq099] term using the Stouffer's method: $\sum w_i M_{ij} / \sqrt{\sum w_i^2}$, where $w_i$ is a weight based on the GWAS sample size for trait $i$, and $M_{ij}$ is the $z$-score for gene $j$. Second, we divided all $z$-scores for each trait $i$ by their sum to reduce the effect of highly polygenic traits: $M_{ij} / \sum M_{ij}$. Finally, we projected this data matrix using Equation (@eq:proj), obtaining $\hat{\mathbf{M}}$ with $n$=3,752 traits and $l$=987 LVs as the input of our clustering pipeline.


In [246]:
par1 = (
    process_paragraph(mod_section_paragraphs[63:67])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace("$$ where", "$$\nwhere")
    .replace("$$ M", "$$\nM")
)
print(par1)

We performed two preprocessing steps on the S-MultiXcan results before the cluster analysis. First, we combined results in $\mathbf{M}$ (with $p$-values converted to $z$-scores, as described before) for traits that mapped to the same Experimental Factor Ontology (EFO) [@doi:10.1093/bioinformatics/btq099] term using the Stouffer's method: 
$$
\sum w_i M_{ij} / \sqrt{\sum w_i^2} 
$$
where $w_i$ is a weight based on the GWAS sample size for trait $i$, and $M_{ij}$ is the $z$-score for gene $j$. Second, we divided all $z$-scores for each trait $i$ by their sum to reduce the effect of highly polygenic traits: 
$$
M_{ij} / \sum M_{ij} 
$$


In [247]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [248]:
display(paragraph_matches[-1])

('methods',
 "We performed two preprocessing steps on the S-MultiXcan results before the cluster analysis. First, we combined results in $\\mathbf{M}$ (with $p$-values converted to $z$-scores, as described before) for traits that mapped to the same Experimental Factor Ontology (EFO) [@doi:10.1093/bioinformatics/btq099] term using the Stouffer's method: $\\sum w_i M_{ij} / \\sqrt{\\sum w_i^2}$, where $w_i$ is a weight based on the GWAS sample size for trait $i$, and $M_{ij}$ is the $z$-score for gene $j$. Second, we divided all $z$-scores for each trait $i$ by their sum to reduce the effect of highly polygenic traits: $M_{ij} / \\sum M_{ij}$. Finally, we projected this data matrix using Equation (@eq:proj), obtaining $\\hat{\\mathbf{M}}$ with $n$=3,752 traits and $l$=987 LVs as the input of our clustering pipeline.",
 "We performed two preprocessing steps on the S-MultiXcan results before the cluster analysis. First, we combined results in $\\mathbf{M}$ (with $p$-values converted to $z$

####  Paragraph 11

In [249]:
par0 = (
    process_paragraph(orig_section_paragraphs[48:51])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":obj_func} where", ":obj_func}\nwhere")
)
print(par0)

A partitioning of $\hat{\mathbf{M}}$ with $n$ traits into $k$ clusters is represented as a label vector $\pi \in \mathbb{N}^n$. Consensus clustering approaches consist of two steps: 1) the generation of an ensemble $\Pi$ with $r$ partitions of the dataset: $\Pi=\{\pi_1, \pi_2, \ldots, \pi_r\}$, and 2) the combination of the ensemble into a consolidated solution defined as: 
$$
\pi^* = \mathrm{arg}\,\underset{\hat{\pi}}{\max} Q(\{ \lvert \mathcal{L}^i \lvert \phi(\hat{\pi}_{\mathcal{L}^i}, \pi_{i \mathcal{L}^i}) \mid i \in \{1,\ldots,r\} \}), 
$$ {#eq:consensus:obj_func}
where $\mathcal{L}^i$ is a set of data indices with known cluster labels for partition $i$, $\phi\colon \mathbb{N}^n \times \mathbb{N}^n \to \mathbb{R}$ is a function that measures the similarity between two partitions, and $Q$ is a measure of central tendency, such as the mean or median. We used the adjusted Rand index (ARI) [@doi:10.1007/BF01908075] for $\phi$ and the median for $Q$. To obtain $\pi^*$, we define a con

In [250]:
par1 = (
    process_paragraph(mod_section_paragraphs[68:71])
    .replace("$$", "\n$$")
    .replace("$$ \\", "$$\n\\")
    .replace(":obj_func} where", ":obj_func}\nwhere")
)
print(par1)

A partitioning of $\hat{\mathbf{M}}$ with $n$ traits into $k$ clusters is represented as a label vector $\pi \in \mathbb{N}^n$. Consensus clustering approaches consist of two steps: 1) the generation of an ensemble $\Pi$ with $r$ partitions of the dataset: $\Pi=\{\pi_1, \pi_2, \ldots, \pi_r\}$, and 2) the combination of the ensemble into a consolidated solution defined as: 
$$
\pi^* = \mathrm{arg}\,\underset{\hat{\pi}}{\max} Q(\{ \lvert \mathcal{L}^i \lvert \phi(\hat{\pi}_{\mathcal{L}^i}, \pi_{i \mathcal{L}^i}) \mid i \in \{1,\ldots,r\} \}), 
$$ {#eq:consensus:obj_func}
where $\mathcal{L}^i$ is a set of data indices with known cluster labels for partition $i$, $\phi\colon \mathbb{N}^n \times \mathbb{N}^n \to \mathbb{R}$ is a function that measures the similarity between two partitions, and $Q$ is a measure of central tendency, such as the mean or median. The adjusted Rand index (ARI) [@doi:10.1007/BF01908075] is used for $\phi$, and the median is used for $Q$. To obtain $\pi^*$, a cons

In [251]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [252]:
display(paragraph_matches[-1])

('methods',
 'A partitioning of $\\hat{\\mathbf{M}}$ with $n$ traits into $k$ clusters is represented as a label vector $\\pi \\in \\mathbb{N}^n$. Consensus clustering approaches consist of two steps: 1) the generation of an ensemble $\\Pi$ with $r$ partitions of the dataset: $\\Pi=\\{\\pi_1, \\pi_2, \\ldots, \\pi_r\\}$, and 2) the combination of the ensemble into a consolidated solution defined as: \n$$\n\\pi^* = \\mathrm{arg}\\,\\underset{\\hat{\\pi}}{\\max} Q(\\{ \\lvert \\mathcal{L}^i \\lvert \\phi(\\hat{\\pi}_{\\mathcal{L}^i}, \\pi_{i \\mathcal{L}^i}) \\mid i \\in \\{1,\\ldots,r\\} \\}), \n$$ {#eq:consensus:obj_func}\nwhere $\\mathcal{L}^i$ is a set of data indices with known cluster labels for partition $i$, $\\phi\\colon \\mathbb{N}^n \\times \\mathbb{N}^n \\to \\mathbb{R}$ is a function that measures the similarity between two partitions, and $Q$ is a measure of central tendency, such as the mean or median. We used the adjusted Rand index (ARI) [@doi:10.1007/BF01908075] for $\\

####  Paragraph 12

In [253]:
par0 = process_paragraph(orig_section_paragraphs[51])
print(par0)

For the ensemble generation step, we used different algorithms to create a highly diverse set of partitions (see Figure @fig:clustering:design) since diversity is an important property for ensembles [@doi:10.1016/j.ins.2016.04.027; @doi:10.1109/TPAMI.2011.84; @doi:10.1016/j.patcog.2014.04.005]. We used three data representations: the raw dataset, its projection into the top 50 principal components, and the embedding learned by UMAP [@arxiv:1802.03426] using 50 components. For each of these, we applied five clustering algorithms covering a wide range of different assumptions on the data structure: $k$-means [@Arthur2007], spectral clustering [@Ng2001], a Gaussian mixture model (GMM), hierarchical clustering, and DBSCAN [@Ester1996]. For $k$-means, spectral clustering and GMM, we specified a range of $k$ between 2 and $\sqrt{n} \approx 60$, and for each $k$ we generated five partitions using random seeds. For hierarchical clustering, for each $k$, we generated four partitions using commo

In [254]:
par1 = (
    process_paragraph(mod_section_paragraphs[71:76])
    .replace("$$", "\n$$")
    .replace("$$ ", "$$\n")
    # .replace(":obj_func} where", ":obj_func}\nwhere")
)
print(par1)

For the ensemble generation step, we utilized various algorithms to create a highly diverse set of partitions (see Figure 1) since diversity is a crucial property for ensembles (Li et al., 2016; Zhang et al., 2011; Wang et al., 2014). We employed three data representations: the raw dataset, its projection into the top 50 principal components, and the embedding learned by UMAP (McInnes et al., 2018) using 50 components. For each of these, we applied five clustering algorithms covering a wide range of different assumptions on the data structure: k-means (Arthur & Vassilvitskii, 2007), spectral clustering (Ng et al., 2001), a Gaussian mixture model (GMM), hierarchical clustering, and DBSCAN (Ester et al., 1996). 
$$
k: \text{Number of clusters} 
$$

$$
\epsilon: \text{Maximum distance between two data points} 
$$

$$
minPts: \text{Minimum number of data points in a neighborhood} 
$$
For k-means, spectral clustering, and GMM, we specified a range of k between 2 and √n ≈ 60, and for each k,

In [255]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [256]:
display(paragraph_matches[-1])

('methods',
 'For the ensemble generation step, we used different algorithms to create a highly diverse set of partitions (see Figure @fig:clustering:design) since diversity is an important property for ensembles [@doi:10.1016/j.ins.2016.04.027; @doi:10.1109/TPAMI.2011.84; @doi:10.1016/j.patcog.2014.04.005]. We used three data representations: the raw dataset, its projection into the top 50 principal components, and the embedding learned by UMAP [@arxiv:1802.03426] using 50 components. For each of these, we applied five clustering algorithms covering a wide range of different assumptions on the data structure: $k$-means [@Arthur2007], spectral clustering [@Ng2001], a Gaussian mixture model (GMM), hierarchical clustering, and DBSCAN [@Ester1996]. For $k$-means, spectral clustering and GMM, we specified a range of $k$ between 2 and $\\sqrt{n} \\approx 60$, and for each $k$ we generated five partitions using random seeds. For hierarchical clustering, for each $k$, we generated four partit

####  Paragraph 13

In [257]:
par0 = process_paragraph(orig_section_paragraphs[52])
print(par0)

Finally, we used spectral clustering on $\mathbf{D}$ to derive the final consensus partitions. $\mathbf{D}$ was first transformed into a similarity matrix by applying an RBF kernel $\mathrm{exp}(-\gamma \mathbf{D}^2)$ using four different values for $\gamma$ that we empirically determined to work best. Therefore, for each $k$ between 2 and 60, we derived four consensus partitions and selected the one that maximized Equation (@eq:consensus:obj_func). We further filtered this set of 59 solutions to keep only those with an ensemble agreement larger than the 75th percentile (Figure @fig:sup:clustering:agreement), leaving a total of 15 final consensus partitions shown in Figure @fig:clustering:tree.


In [258]:
par1 = process_paragraph(mod_section_paragraphs[76])
print(par1)

Finally, we used spectral clustering on matrix $\mathbf{D}$ to derive the final consensus partitions. Matrix $\mathbf{D}$ was first transformed into a similarity matrix by applying a Radial Basis Function (RBF) kernel $\mathrm{exp}(-\gamma \mathbf{D}^2)$ using four different values for $\gamma$ that were empirically determined to work best. Therefore, for each value of $k$ between 2 and 60, we derived four consensus partitions and selected the one that maximized Equation (1). We further filtered this set of 59 solutions to keep only those with an ensemble agreement larger than the 75th percentile (Figure 1), leaving a total of 15 final consensus partitions as shown in Figure 2.


In [259]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [260]:
display(paragraph_matches[-1])

('methods',
 'Finally, we used spectral clustering on $\\mathbf{D}$ to derive the final consensus partitions. $\\mathbf{D}$ was first transformed into a similarity matrix by applying an RBF kernel $\\mathrm{exp}(-\\gamma \\mathbf{D}^2)$ using four different values for $\\gamma$ that we empirically determined to work best. Therefore, for each $k$ between 2 and 60, we derived four consensus partitions and selected the one that maximized Equation (@eq:consensus:obj_func). We further filtered this set of 59 solutions to keep only those with an ensemble agreement larger than the 75th percentile (Figure @fig:sup:clustering:agreement), leaving a total of 15 final consensus partitions shown in Figure @fig:clustering:tree.',
 'Finally, we used spectral clustering on matrix $\\mathbf{D}$ to derive the final consensus partitions. Matrix $\\mathbf{D}$ was first transformed into a similarity matrix by applying a Radial Basis Function (RBF) kernel $\\mathrm{exp}(-\\gamma \\mathbf{D}^2)$ using four d

####  Paragraph 14

In [261]:
par0 = process_paragraph(orig_section_paragraphs[53])
print(par0)

The input data in our clustering pipeline undergoes several linear and nonlinear transformations, including PCA, UMAP and the ensemble transformation using the EAC paradigm (distance matrix $\mathbf{D}$). Although consensus clustering has clear advantages for biological data [@pmid:27303057], this set of data transformations complicates the interpretation of results. To circumvent this, we used a supervised learning approach to detect which gene modules/LVs are the most important for each cluster of traits (Figure {@fig:clustering:design}b). Note that we did not use this supervised model for prediction but only to learn which features (LVs) were most discriminative for each cluster. For this, we used the highest resolution partition ($k$=29, although any could be used) to train a decision tree model using each of the clusters as labels and the projected data $\hat{\mathbf{M}}$ as the training samples. For each $k$, we built a set of binary labels with the current cluster's traits as th

In [262]:
par1 = process_paragraph(
    [
        mod_section_paragraphs[78],
        mod_section_paragraphs[80],
    ]
)
print(par1)

The input data in our clustering pipeline undergoes several linear and nonlinear transformations, including Principal Component Analysis (PCA), Uniform Manifold Approximation and Projection (UMAP), and the ensemble transformation using the Ensemble Aggregation Clustering (EAC) paradigm (distance matrix $\mathbf{D}$). Although consensus clustering has clear advantages for biological data (Wilkerson et al., 2016), this set of data transformations complicates the interpretation of results. To circumvent this, we used a supervised learning approach to detect which gene modules/Latent Variables (LVs) are the most important for each cluster of traits (Figure 1b). Note that we did not use this supervised model for prediction but only to learn which features (LVs) were most discriminative for each cluster. For this, we used the highest resolution partition ($k$=29, although any could be used) to train a decision tree model using each of the clusters as labels and the projected data $\hat{\math

In [263]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [264]:
display(paragraph_matches[-1])

('methods',
 "The input data in our clustering pipeline undergoes several linear and nonlinear transformations, including PCA, UMAP and the ensemble transformation using the EAC paradigm (distance matrix $\\mathbf{D}$). Although consensus clustering has clear advantages for biological data [@pmid:27303057], this set of data transformations complicates the interpretation of results. To circumvent this, we used a supervised learning approach to detect which gene modules/LVs are the most important for each cluster of traits (Figure {@fig:clustering:design}b). Note that we did not use this supervised model for prediction but only to learn which features (LVs) were most discriminative for each cluster. For this, we used the highest resolution partition ($k$=29, although any could be used) to train a decision tree model using each of the clusters as labels and the projected data $\\hat{\\mathbf{M}}$ as the training samples. For each $k$, we built a set of binary labels with the current clust

####  Paragraph 15

In [265]:
par0 = process_paragraph(orig_section_paragraphs[56])
print(par0)

**Cell culture.** HepG2 cells were obtained from ATCC (ATCC® HB-8065™), and maintained in Eagle's Minimum Essential Medium with L-Glutamine (EMEM, Cat. 112-018-101, Quality Biology) supplemented with 10% Fetal Bovine Serum (FBS, Gibco, Cat.16000-044), and 1% Pen/Strep (Gibco, Cat.15140-122). Cells were kept at 37oC in a humidity-controlled incubator with 5% CO2, and were maintained at a density not exceeding more than 80% confluency in Collagen-I coated flasks.


In [266]:
par1 = process_paragraph(mod_section_paragraphs[83])
print(par1)

**Cell culture.** HepG2 cells were obtained from ATCC (ATCC® HB-8065™) and maintained in Eagle's Minimum Essential Medium with L-Glutamine (EMEM, Cat. 112-018-101, Quality Biology) supplemented with 10% Fetal Bovine Serum (FBS, Gibco, Cat.16000-044) and 1% Pen/Strep (Gibco, Cat.15140-122). Cells were kept at 37°C in a humidity-controlled incubator with 5% CO2 and were maintained at a density not exceeding 80% confluency in Collagen-I coated flasks.


In [267]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [268]:
display(paragraph_matches[-1])

('methods',
 "**Cell culture.** HepG2 cells were obtained from ATCC (ATCC® HB-8065™), and maintained in Eagle's Minimum Essential Medium with L-Glutamine (EMEM, Cat. 112-018-101, Quality Biology) supplemented with 10% Fetal Bovine Serum (FBS, Gibco, Cat.16000-044), and 1% Pen/Strep (Gibco, Cat.15140-122). Cells were kept at 37oC in a humidity-controlled incubator with 5% CO2, and were maintained at a density not exceeding more than 80% confluency in Collagen-I coated flasks.",
 "**Cell culture.** HepG2 cells were obtained from ATCC (ATCC® HB-8065™) and maintained in Eagle's Minimum Essential Medium with L-Glutamine (EMEM, Cat. 112-018-101, Quality Biology) supplemented with 10% Fetal Bovine Serum (FBS, Gibco, Cat.16000-044) and 1% Pen/Strep (Gibco, Cat.15140-122). Cells were kept at 37°C in a humidity-controlled incubator with 5% CO2 and were maintained at a density not exceeding 80% confluency in Collagen-I coated flasks.")

####  Paragraph 16

In [269]:
par0 = process_paragraph(orig_section_paragraphs[57])
print(par0)

**Genome-wide lentiviral pooled CRISPR-Cas9 library.** 3rd lentiviral generation, Broad GPP genome-wide Human Brunello CRISPR knockout Pooled library was provided by David Root and John Doench from Addgene (Cat. 73179-LV), and was used for HepG2 cell transduction. It consists of 76,441 sgRNAs, and targets 19,114 genes in the human genome with an average of 4 sgRNAs per gene. Each 20nt sgRNA cassette was inserted into the lentiCRIS-PRv2 backbone between U6 promoter and gRNA scaffold. Through cell transduction, the lentiviral vectors which encode Cas9 were used to deliver the sgRNA cassette containing plasmids into cells during cell replication. Unsuccessful transduced cells were excluded through puromycin selection.


In [270]:
par1 = process_paragraph(mod_section_paragraphs[84])
print(par1)

**Genome-wide lentiviral pooled CRISPR-Cas9 library.** The 3rd generation lentiviral library, known as Broad GPP genome-wide Human Brunello CRISPR knockout Pooled library, was obtained from Addgene (Cat. 73179-LV) courtesy of David Root and John Doench. This library was utilized for transduction of HepG2 cells and comprises 76,441 sgRNAs targeting 19,114 genes in the human genome, with an average of 4 sgRNAs per gene. Each 20 nucleotide sgRNA sequence was integrated into the lentiCRIS-PRv2 backbone between the U6 promoter and gRNA scaffold. Cas9-encoding lentiviral vectors were employed for the delivery of plasmids containing the sgRNA cassette into cells during replication. Cells that were not successfully transduced were eliminated through puromycin selection.


In [271]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [272]:
display(paragraph_matches[-1])

('methods',
 '**Genome-wide lentiviral pooled CRISPR-Cas9 library.** 3rd lentiviral generation, Broad GPP genome-wide Human Brunello CRISPR knockout Pooled library was provided by David Root and John Doench from Addgene (Cat. 73179-LV), and was used for HepG2 cell transduction. It consists of 76,441 sgRNAs, and targets 19,114 genes in the human genome with an average of 4 sgRNAs per gene. Each 20nt sgRNA cassette was inserted into the lentiCRIS-PRv2 backbone between U6 promoter and gRNA scaffold. Through cell transduction, the lentiviral vectors which encode Cas9 were used to deliver the sgRNA cassette containing plasmids into cells during cell replication. Unsuccessful transduced cells were excluded through puromycin selection.',
 '**Genome-wide lentiviral pooled CRISPR-Cas9 library.** The 3rd generation lentiviral library, known as Broad GPP genome-wide Human Brunello CRISPR knockout Pooled library, was obtained from Addgene (Cat. 73179-LV) courtesy of David Root and John Doench. Thi

####  Paragraph 17

In [273]:
par0 = process_paragraph(orig_section_paragraphs[58])
print(par0)

**Lentiviral titer determination.** No-spin lentiviral transduction was utilized for the screen. In a Collagen-I coated 6-wells plate, approximate 2.5 M cells were seeded each well in the presence of 8ug/ml polybrene (Millipore Sigma, Cat. TR-1003 G), and a different titrated virus volume (e.g., 0, 50, 100, 200, 250, and 400ul) was assigned to each well. EMEM complete media was added to make the final volume of 1.24ml. 16-18hrs post-transduction, virus/polybrene-containing media was removed from each well. Cells were washed twice with 1x DPBS and replaced with fresh EMEM. At 24h, cells in each well were trypsinized, diluted (e.g.,1:10), and seeded in pairs of wells of 6-well plates. At 60hr post-transduction, cell media in each well was replaced with fresh EMEM. 2ug/ml of puromycin (Gibco, Cat. A1113803) was added to one well out of the pair. 2-5 days after puromycin selection, or the 0 virus well treated with puromycin had no survival of cells, cells in both wells with/without puromyc

In [274]:
par1 = process_paragraph(mod_section_paragraphs[85])
print(par1)

**Lentiviral titer determination.** No-spin lentiviral transduction was used for the screen. Approximately 2.5 million cells were seeded in each well of a Collagen-I coated 6-well plate in the presence of 8 µg/ml polybrene (Millipore Sigma, Cat. TR-1003 G). Different titrated virus volumes (e.g., 0, 50, 100, 200, 250, and 400 µl) were assigned to each well. EMEM complete media was added to achieve a final volume of 1.24 ml. Sixteen to eighteen hours post-transduction, the virus/polybrene-containing media was removed from each well. Cells were washed twice with 1x DPBS and fresh EMEM was added. At 24 hours, cells in each well were trypsinized, diluted (e.g., 1:10), and seeded in pairs of wells in 6-well plates. At 60 hours post-transduction, the cell media in each well was replaced with fresh EMEM. Two µg/ml of puromycin (Gibco, Cat. A1113803) was added to one well out of the pair. Two to five days after puromycin selection, or if the 0 virus well treated with puromycin showed no cell s

In [275]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [276]:
display(paragraph_matches[-1])

('methods',
 "**Lentiviral titer determination.** No-spin lentiviral transduction was utilized for the screen. In a Collagen-I coated 6-wells plate, approximate 2.5 M cells were seeded each well in the presence of 8ug/ml polybrene (Millipore Sigma, Cat. TR-1003 G), and a different titrated virus volume (e.g., 0, 50, 100, 200, 250, and 400ul) was assigned to each well. EMEM complete media was added to make the final volume of 1.24ml. 16-18hrs post-transduction, virus/polybrene-containing media was removed from each well. Cells were washed twice with 1x DPBS and replaced with fresh EMEM. At 24h, cells in each well were trypsinized, diluted (e.g.,1:10), and seeded in pairs of wells of 6-well plates. At 60hr post-transduction, cell media in each well was replaced with fresh EMEM. 2ug/ml of puromycin (Gibco, Cat. A1113803) was added to one well out of the pair. 2-5 days after puromycin selection, or the 0 virus well treated with puromycin had no survival of cells, cells in both wells with/w

####  Paragraph 18

In [277]:
par0 = process_paragraph(orig_section_paragraphs[59])
print(par0)

**Lentiviral Transduction in HepG2 Using Brunello CRISPR Knockout Pooled Library.** In order to achieve a coverage (representation) of at least 500 cells per sgRNA, and at an MOI between 0.3-0.4 to ensure 95% of infected cells get only one viral particle per cell, ~200M cells were initiated for the screen. Transduction was carried out in a similar fashion as described above. Briefly, 2.5M cells were seeded in each well of 14 6-well plates, along with 8ug/ml of polybrene. A volume of 120ul of the virus was added to each experimental well. 18hrs post-transduction, virus/PB mix medium was removed, and cells in each well were collected, counted, and pooled into T175 flasks. At 60hr post-transduction, 2ug/ml of puromycin was added to each flask. Mediums were changed every two days with fresh EMEM, topped with 2ug/ml puromycin. Seven days after puromycin selection, cells were collected, pooled, counted, and replated.


In [278]:
par1 = process_paragraph(mod_section_paragraphs[87])
print(par1)

Lentiviral transduction was performed in HepG2 cells using the Brunello CRISPR knockout pooled library to ensure a coverage of at least 500 cells per single-guide RNA (sgRNA). An MOI between 0.3-0.4 was maintained to guarantee that 95% of infected cells received only one viral particle per cell. Approximately 200 million cells were used for the screen. The transduction process followed a similar protocol as previously described. Specifically, 2.5 million cells were seeded in each well of 14 6-well plates, supplemented with 8 μg/ml of polybrene. Subsequently, 120 μl of the virus was added to each experimental well. After 18 hours post-transduction, the virus/polybrene mix was removed, and cells from each well were collected, counted, and pooled into T175 flasks. At 60 hours post-transduction, 2 μg/ml of puromycin was added to each flask. The medium was changed every two days with fresh EMEM containing 2 μg/ml puromycin. Seven days after puromycin selection, cells were collected, pooled,

In [279]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [280]:
display(paragraph_matches[-1])

('methods',
 '**Lentiviral Transduction in HepG2 Using Brunello CRISPR Knockout Pooled Library.** In order to achieve a coverage (representation) of at least 500 cells per sgRNA, and at an MOI between 0.3-0.4 to ensure 95% of infected cells get only one viral particle per cell, ~200M cells were initiated for the screen. Transduction was carried out in a similar fashion as described above. Briefly, 2.5M cells were seeded in each well of 14 6-well plates, along with 8ug/ml of polybrene. A volume of 120ul of the virus was added to each experimental well. 18hrs post-transduction, virus/PB mix medium was removed, and cells in each well were collected, counted, and pooled into T175 flasks. At 60hr post-transduction, 2ug/ml of puromycin was added to each flask. Mediums were changed every two days with fresh EMEM, topped with 2ug/ml puromycin. Seven days after puromycin selection, cells were collected, pooled, counted, and replated.',
 'Lentiviral transduction was performed in HepG2 cells usin

####  Paragraph 19

In [281]:
par0 = process_paragraph(orig_section_paragraphs[60])
print(par0)

**Fluorescent dye staining.** 9 days after puromycin selection, cells were assigned to 2 groups. 20-30M cells were collected as Unsorted Control. The cell pellet was spun down at 500 x g for 5min at 4oC. The dry pellet was kept at -80oC for further genomic DNA isolation. The rest of the cells (approximately 200M) were kept in 100mm dishes and stained with a fluorescent dye (LipidSpotTM 488, Biotium, Cat. 70065-T). In Brief, LipidSpot 488 was diluted to 1:100 with DPBS. 4ml of staining solution was used for each dish and incubated at 37oC for 30min. Cell images were captured through fluorescent microscope EVOS for GFP signal detection (Figure @fig:sup:crispr:fig1).


In [282]:
par1 = process_paragraph(mod_section_paragraphs[90])
print(par1)

**Fluorescent dye staining.** Nine days after puromycin selection, cells were divided into two groups. Twenty to thirty million cells were collected as the Unsorted Control. The cell pellet was centrifuged at 500 x g for 5 minutes at 4°C. The dry pellet was stored at -80°C for subsequent genomic DNA isolation. The remaining cells (approximately 200 million) were cultured in 100 mm dishes and stained with a fluorescent dye (LipidSpotTM 488, Biotium, Cat. 70065-T). In brief, LipidSpot 488 was diluted to 1:100 with DPBS. Four milliliters of the staining solution were added to each dish and incubated at 37°C for 30 minutes. Cell images were captured using a fluorescent microscope EVOS for GFP signal detection (Figure 1).


In [283]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [284]:
display(paragraph_matches[-1])

('methods',
 '**Fluorescent dye staining.** 9 days after puromycin selection, cells were assigned to 2 groups. 20-30M cells were collected as Unsorted Control. The cell pellet was spun down at 500 x g for 5min at 4oC. The dry pellet was kept at -80oC for further genomic DNA isolation. The rest of the cells (approximately 200M) were kept in 100mm dishes and stained with a fluorescent dye (LipidSpotTM 488, Biotium, Cat. 70065-T). In Brief, LipidSpot 488 was diluted to 1:100 with DPBS. 4ml of staining solution was used for each dish and incubated at 37oC for 30min. Cell images were captured through fluorescent microscope EVOS for GFP signal detection (Figure @fig:sup:crispr:fig1).',
 '**Fluorescent dye staining.** Nine days after puromycin selection, cells were divided into two groups. Twenty to thirty million cells were collected as the Unsorted Control. The cell pellet was centrifuged at 500 x g for 5 minutes at 4°C. The dry pellet was stored at -80°C for subsequent genomic DNA isolatio

####  Paragraph 20

In [285]:
par0 = process_paragraph(orig_section_paragraphs[61])
print(par0)

**Fluorescence-activated cell sorting (FACS).** Cells were immediately collected into 50ml tubes (From this point on, keep cells cold), and spun at 500 x g for 5min at 4oC. After DPBS wash, cell pellets were resuspended with FACS Sorting Buffer (1x DPBS without Ca2+/Mg2+, 2.5mM EDTA, 25mM HEPES, 1% BSA. The solution was filter sterilized, and kept at 4oC), with gentle pipetting to make single cells. The cell solution was then filtered through a cell strainer (Falcon, Cat. 352235) and was kept on ice, protected from light. Collected cells were sorted on FACSJazz. 100um nozzle was used for sorting. ~20% of each GFP-High and GFP-Low (Figure @fig:sup:crispr:fig2) were collected into 15ml tubes. After sorting, cells were immediately spun down. Pellets were kept at -80oC for further genomic DNA isolation.


In [286]:
par1 = process_paragraph(mod_section_paragraphs[91])
print(par1)

**Fluorescence-activated cell sorting (FACS).** Cells were immediately collected into 50 mL tubes (from this point on, keep cells cold) and spun at 500 x g for 5 minutes at 4°C. After DPBS wash, cell pellets were resuspended with FACS Sorting Buffer (1x DPBS without Ca2+/Mg2+, 2.5 mM EDTA, 25 mM HEPES, 1% BSA). The solution was filter sterilized and kept at 4°C, with gentle pipetting to ensure single cells. The cell solution was then filtered through a cell strainer (Falcon, Cat. 352235) and kept on ice, protected from light. Collected cells were sorted on FACSJazz using a 100 μm nozzle for sorting. Approximately 20% of each GFP-High and GFP-Low population (Figure 2) were collected into 15 mL tubes. After sorting, cells were immediately spun down, and pellets were kept at -80°C for further genomic DNA isolation.


In [287]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [288]:
display(paragraph_matches[-1])

('methods',
 '**Fluorescence-activated cell sorting (FACS).** Cells were immediately collected into 50ml tubes (From this point on, keep cells cold), and spun at 500 x g for 5min at 4oC. After DPBS wash, cell pellets were resuspended with FACS Sorting Buffer (1x DPBS without Ca2+/Mg2+, 2.5mM EDTA, 25mM HEPES, 1% BSA. The solution was filter sterilized, and kept at 4oC), with gentle pipetting to make single cells. The cell solution was then filtered through a cell strainer (Falcon, Cat. 352235) and was kept on ice, protected from light. Collected cells were sorted on FACSJazz. 100um nozzle was used for sorting. ~20% of each GFP-High and GFP-Low (Figure @fig:sup:crispr:fig2) were collected into 15ml tubes. After sorting, cells were immediately spun down. Pellets were kept at -80oC for further genomic DNA isolation.',
 '**Fluorescence-activated cell sorting (FACS).** Cells were immediately collected into 50 mL tubes (from this point on, keep cells cold) and spun at 500 x g for 5 minutes a

####  Paragraph 21

In [289]:
par0 = process_paragraph(orig_section_paragraphs[62])
print(par0)

**Genomic DNA isolation and verification.** Three conditions of Genomic DNA (Un-Sorted Control, lentiV2 GFP-High, and lentiV2 GFP-Low) were extracted using QIAamp DNA Blood Mini Kit (Qiagen, Cat.51104), followed by UV Spectroscopy (Nanodrop) to access the quality and quantity of the gDNA. A total of 80-160ug of gDNA was isolated for each condition. sgRNA cassette and lentiviral specific transgene in isolated gDNA were verified through PCR (Figure @fig:sup:crispr:fig3).


In [290]:
par1 = process_paragraph(mod_section_paragraphs[92])
print(par1)

**Genomic DNA isolation and verification.** Three conditions of Genomic DNA (Un-Sorted Control, lentiV2 GFP-High, and lentiV2 GFP-Low) were extracted using the QIAamp DNA Blood Mini Kit (Qiagen, Cat.51104), followed by UV Spectroscopy (Nanodrop) to assess the quality and quantity of the gDNA. A total of 80-160 µg of gDNA was isolated for each condition. The sgRNA cassette and lentiviral specific transgene in the isolated gDNA were verified through PCR (Figure 3).


In [291]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [292]:
display(paragraph_matches[-1])

('methods',
 '**Genomic DNA isolation and verification.** Three conditions of Genomic DNA (Un-Sorted Control, lentiV2 GFP-High, and lentiV2 GFP-Low) were extracted using QIAamp DNA Blood Mini Kit (Qiagen, Cat.51104), followed by UV Spectroscopy (Nanodrop) to access the quality and quantity of the gDNA. A total of 80-160ug of gDNA was isolated for each condition. sgRNA cassette and lentiviral specific transgene in isolated gDNA were verified through PCR (Figure @fig:sup:crispr:fig3).',
 '**Genomic DNA isolation and verification.** Three conditions of Genomic DNA (Un-Sorted Control, lentiV2 GFP-High, and lentiV2 GFP-Low) were extracted using the QIAamp DNA Blood Mini Kit (Qiagen, Cat.51104), followed by UV Spectroscopy (Nanodrop) to assess the quality and quantity of the gDNA. A total of 80-160 µg of gDNA was isolated for each condition. The sgRNA cassette and lentiviral specific transgene in the isolated gDNA were verified through PCR (Figure 3).')

####  Paragraph 22

In [293]:
par0 = process_paragraph(orig_section_paragraphs[63])
print(par0)

**Illumina libraries generation and sequencing.** The fragment containing sgRNA cassette was amplified using P5 /P7 primers, as indicated in [@pmid:26780180], and primer sequences were adapted from Broad Institute protocol (Figure @fig:sup:crispr:table1). Stagger sequence (0-8nt) was included in P5 and 8bp uniquely barcoded sequence in P7. Primers were synthesized through Integrated DNA Technologies (IDT), and each primer was PAGE purified. 32 PCR reactions were set up for each condition. Each 100ul PCR reaction consists of roughly 5ug of gDNA, 5ul of each 10uM P5 and P7. ExTaq DNA Polymerase (TaKaRa, Cat. RR001A) was used to amplify the amplicon. PCR Thermal Cycler Parameters set as Initial at 95oC for 1min; followed by 24 cycles of Denaturation at 94oC for 30 seconds, Annealing at 52.5oC for 30 seconds, Extension at 72oC for 30 seconds. A final Elongation at 72oC for 10 minutes. 285bp-293bp PCR products were expected (Figure @fig:sup:crispr:fig4 A). PCR products within the same condi

In [294]:
par1 = process_paragraph(mod_section_paragraphs[94])
print(par1)

**Illumina libraries generation and sequencing.** The fragment containing the sgRNA cassette was amplified using P5/P7 primers, as indicated in (Doench et al., 2016), and primer sequences were adapted from the Broad Institute protocol (Figure 1). The stagger sequence (0-8 nt) was included in P5, and an 8 bp uniquely barcoded sequence was included in P7. Primers were synthesized through Integrated DNA Technologies (IDT), and each primer was PAGE purified. 32 PCR reactions were set up for each condition. Each 100 μl PCR reaction consisted of roughly 5 μg of gDNA, 5 μl of each 10 μM P5 and P7 primers. ExTaq DNA Polymerase (TaKaRa, Cat. RR001A) was used to amplify the amplicon. The PCR thermal cycler parameters were set as follows: initial denaturation at 95°C for 1 min; followed by 24 cycles of denaturation at 94°C for 30 seconds, annealing at 52.5°C for 30 seconds, extension at 72°C for 30 seconds. A final elongation step at 72°C for 10 minutes was performed. PCR products of 285-293 bp w

In [295]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [296]:
display(paragraph_matches[-1])

('methods',
 '**Illumina libraries generation and sequencing.** The fragment containing sgRNA cassette was amplified using P5 /P7 primers, as indicated in [@pmid:26780180], and primer sequences were adapted from Broad Institute protocol (Figure @fig:sup:crispr:table1). Stagger sequence (0-8nt) was included in P5 and 8bp uniquely barcoded sequence in P7. Primers were synthesized through Integrated DNA Technologies (IDT), and each primer was PAGE purified. 32 PCR reactions were set up for each condition. Each 100ul PCR reaction consists of roughly 5ug of gDNA, 5ul of each 10uM P5 and P7. ExTaq DNA Polymerase (TaKaRa, Cat. RR001A) was used to amplify the amplicon. PCR Thermal Cycler Parameters set as Initial at 95oC for 1min; followed by 24 cycles of Denaturation at 94oC for 30 seconds, Annealing at 52.5oC for 30 seconds, Extension at 72oC for 30 seconds. A final Elongation at 72oC for 10 minutes. 285bp-293bp PCR products were expected (Figure @fig:sup:crispr:fig4 A). PCR products within 

####  Paragraph 23

In [297]:
par0 = process_paragraph(orig_section_paragraphs[67])
print(par0)

The data used from PhenomeXcan, LINCS L1000, and MultiPLIER are publicly available. All significant results reported for the eMERGE and Penn Medicine BioBank (PMBB) phenome-wide TWAS are contained in [@doi:10.1101/2021.10.21.21265225]. The individual-level PMBB raw datasets can not be made publicly available due to institutional privacy policy. Please contact Penn Medicine Biobank ([https://pmbb.med.upenn.edu/pmbb/](https://pmbb.med.upenn.edu/pmbb/)) for requests of access to data. eMERGE network phase III data is available on dbGAP (Accession: phs001584.v2.p2).


In [298]:
par1 = process_paragraph(mod_section_paragraphs[98])
print(par1)

The data utilized from PhenomeXcan, LINCS L1000, and MultiPLIER are publicly available. All significant results reported for the eMERGE and Penn Medicine BioBank (PMBB) phenome-wide TWAS are contained in the study by Gusev et al. (2021). The individual-level PMBB raw datasets cannot be made publicly available due to institutional privacy policy. For requests for access to data, please contact the Penn Medicine Biobank at [https://pmbb.med.upenn.edu/pmbb/](https://pmbb.med.upenn.edu/pmbb/). The eMERGE network phase III data is accessible on dbGAP (Accession: phs001584.v2.p2).


In [299]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [300]:
display(paragraph_matches[-1])

('methods',
 'The data used from PhenomeXcan, LINCS L1000, and MultiPLIER are publicly available. All significant results reported for the eMERGE and Penn Medicine BioBank (PMBB) phenome-wide TWAS are contained in [@doi:10.1101/2021.10.21.21265225]. The individual-level PMBB raw datasets can not be made publicly available due to institutional privacy policy. Please contact Penn Medicine Biobank ([https://pmbb.med.upenn.edu/pmbb/](https://pmbb.med.upenn.edu/pmbb/)) for requests of access to data. eMERGE network phase III data is available on dbGAP (Accession: phs001584.v2.p2).',
 'The data utilized from PhenomeXcan, LINCS L1000, and MultiPLIER are publicly available. All significant results reported for the eMERGE and Penn Medicine BioBank (PMBB) phenome-wide TWAS are contained in the study by Gusev et al. (2021). The individual-level PMBB raw datasets cannot be made publicly available due to institutional privacy policy. For requests for access to data, please contact the Penn Medicine

####  Paragraph 24

In [301]:
par0 = process_paragraph(orig_section_paragraphs[70])
print(par0)

For the CRISPR screening, we used FlowJo v10.7 and FACS Jazz Software v1.1. For data analysis, we used Python 3.8 and R 3.6 with several computational packages. The main Python packages used were: Jupyter Lab (2.2), pandas (1.1), matplotlib (3.3), seaborn (0.11), numpy (1.19), scipy (1.5), scikit-learn (0.23), and umap-learn (0.4). The main R packages were: Bioconductor (3.10), clusterProfiler (3.14), clustree (0.4), and fgsea (1.17). We also developed several scripts and notebooks which are published under an open-source license. We documented all the steps necessary to carry out all the analyses. We also provide a Docker image to use the same runtime environment we used, and a demo to quickly test the methods on real data.


In [302]:
par1 = process_paragraph(mod_section_paragraphs[104])
print(par1)

For the CRISPR screening, FlowJo v10.7 and FACS Jazz Software v1.1 were utilized. Data analysis was conducted using Python 3.8 and R 3.6 along with various computational packages. The primary Python packages included Jupyter Lab (2.2), pandas (1.1), matplotlib (3.3), seaborn (0.11), numpy (1.19), scipy (1.5), scikit-learn (0.23), and umap-learn (0.4). In R, the main packages used were Bioconductor (3.10), clusterProfiler (3.14), clustree (0.4), and fgsea (1.17). Additionally, several scripts and notebooks were developed and made available under an open-source license. Detailed documentation outlining all analysis steps was provided. A Docker image was also made accessible to replicate the same runtime environment, along with a demo for quick testing of the methods on real data.


In [303]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [304]:
display(paragraph_matches[-1])

('methods',
 'For the CRISPR screening, we used FlowJo v10.7 and FACS Jazz Software v1.1. For data analysis, we used Python 3.8 and R 3.6 with several computational packages. The main Python packages used were: Jupyter Lab (2.2), pandas (1.1), matplotlib (3.3), seaborn (0.11), numpy (1.19), scipy (1.5), scikit-learn (0.23), and umap-learn (0.4). The main R packages were: Bioconductor (3.10), clusterProfiler (3.14), clustree (0.4), and fgsea (1.17). We also developed several scripts and notebooks which are published under an open-source license. We documented all the steps necessary to carry out all the analyses. We also provide a Docker image to use the same runtime environment we used, and a demo to quickly test the methods on real data.',
 'For the CRISPR screening, FlowJo v10.7 and FACS Jazz Software v1.1 were utilized. Data analysis was conducted using Python 3.8 and R 3.6 along with various computational packages. The primary Python packages included Jupyter Lab (2.2), pandas (1.1

## Supplementary material

In [305]:
section_name = "supplementary material"

In [306]:
pr_filename = pr_files[8].filename
assert "supplementary" in pr_filename
print(pr_filename)

content/50.00.supplementary_material.md


### Original

In [307]:
# get content
orig_section_content = repo.get_contents(pr_filename, pr_prev).decoded_content.decode(
    "utf-8"
)
print(orig_section_content[:50])

\clearpage

## Supplementary information {.page_br


In [308]:
# split by paragraph
orig_section_paragraphs = orig_section_content.split("\n\n")
display(len(orig_section_paragraphs))

142

### Modified

In [309]:
# get content
mod_section_content = repo.get_contents(pr_filename, pr_curr).decoded_content.decode(
    "utf-8"
)
print(mod_section_content[:50])

\clearpage

## Supplementary information {.page_br


In [310]:
# split by paragraph
mod_section_paragraphs = mod_section_content.split("\n\n")
display(len(mod_section_paragraphs))

142

### Match

In [311]:
orig_section_paragraphs[0]

'\\clearpage'

In [312]:
mod_section_paragraphs[0]

'\\clearpage'

####  Paragraph 00

In [313]:
par0 = process_paragraph(orig_section_paragraphs[3])
print(par0)

We assessed our GLS model type I error rates (proportion of $p$-values below 0.05) and calibration using a null model of random traits and genotype data from 1000 Genomes Phase III. We selected 312 individuals with European ancestry, and then analyzed 1,000 traits drawn from a standard normal distribution $\mathcal{N}(0,1)$. We ran all the standard procedures for the TWAS approaches (S-PrediXcan and S-MultiXcan), including: 1) a standard GWAS using linear regression under an additive genetic model, 2) different GWAS processing steps, including harmonization and imputation procedures as defined in [@doi:10.1002/gepi.22346], 3) S-PrediXcan and S-MultiXcan analyses. Below we provide details for each of these steps.


In [314]:
par1 = process_paragraph(mod_section_paragraphs[3])
print(par1)

We tested the accuracy of our model by looking at how often $p$-values were below 0.05, and how well it matched a null model with random traits and genotype data from the 1000 Genomes Phase III. We chose 312 people of European descent and examined 1,000 traits that followed a standard normal distribution ($\mathcal{N}(0,1)$). We followed the usual steps for TWAS methods (S-PrediXcan and S-MultiXcan), which included: 1) a standard GWAS using linear regression with an additive genetic model, 2) various GWAS processing procedures like harmonization and imputation as described in a previous study [@doi:10.1002/gepi.22346], and 3) S-PrediXcan and S-MultiXcan analyses. The specifics of each step are outlined below.


In [315]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [316]:
display(paragraph_matches[-1])

('supplementary material',
 'We assessed our GLS model type I error rates (proportion of $p$-values below 0.05) and calibration using a null model of random traits and genotype data from 1000 Genomes Phase III. We selected 312 individuals with European ancestry, and then analyzed 1,000 traits drawn from a standard normal distribution $\\mathcal{N}(0,1)$. We ran all the standard procedures for the TWAS approaches (S-PrediXcan and S-MultiXcan), including: 1) a standard GWAS using linear regression under an additive genetic model, 2) different GWAS processing steps, including harmonization and imputation procedures as defined in [@doi:10.1002/gepi.22346], 3) S-PrediXcan and S-MultiXcan analyses. Below we provide details for each of these steps.',
 'We tested the accuracy of our model by looking at how often $p$-values were below 0.05, and how well it matched a null model with random traits and genotype data from the 1000 Genomes Phase III. We chose 312 people of European descent and exami

####  Paragraph 01

In [317]:
par0 = process_paragraph(orig_section_paragraphs[4])
print(par0)

**Step 1 - GWAS**. We performed standard QC procedures such as filtering out variants with missing call rates eexceeding 0.01, MAF below 1% or MAC below 20, and HWE below 1e-6, and removing samples with high sex-discrepancy and high-relatedness (first and second degree). We included sex and the top 20 principal components as covariates, performing the association test on 5,923,554 variants across all 1,000 random phenotypes.


In [318]:
par1 = process_paragraph(mod_section_paragraphs[4])
print(par1)

In the first step of our study, we conducted a genome-wide association study (GWAS). We followed standard quality control procedures, which involved filtering out variants with missing call rates above 0.01, minor allele frequencies below 1%, minor allele counts below 20, and Hardy-Weinberg equilibrium p-values below 1e-6. We also removed samples that showed high discrepancies in sex or were closely related (up to second degree). Additionally, we included sex and the top 20 principal components as covariates in our analysis. The association test was performed on a total of 5,923,554 variants across all 1,000 randomly selected phenotypes.


In [319]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [320]:
display(paragraph_matches[-1])

('supplementary material',
 '**Step 1 - GWAS**. We performed standard QC procedures such as filtering out variants with missing call rates eexceeding 0.01, MAF below 1% or MAC below 20, and HWE below 1e-6, and removing samples with high sex-discrepancy and high-relatedness (first and second degree). We included sex and the top 20 principal components as covariates, performing the association test on 5,923,554 variants across all 1,000 random phenotypes.',
 'In the first step of our study, we conducted a genome-wide association study (GWAS). We followed standard quality control procedures, which involved filtering out variants with missing call rates above 0.01, minor allele frequencies below 1%, minor allele counts below 20, and Hardy-Weinberg equilibrium p-values below 1e-6. We also removed samples that showed high discrepancies in sex or were closely related (up to second degree). Additionally, we included sex and the top 20 principal components as covariates in our analysis. The ass

####  Paragraph 02

In [321]:
par0 = process_paragraph(orig_section_paragraphs[5])
print(par0)

**Step 2 - GWAS processing**. These steps include harmonization of GWAS and imputation of $z$-scores, which are part of the TWAS pipeline and are needed in order to ensure an acceptable overlap with SNPs in prediction models. The scripts to run these steps are available in [@url:https://github.com/hakyimlab/summary-gwas-imputation]. These procedures were run for all 1,000 random phenotypes and generated a total number of 8,325,729 variants, including those with original and imputed $z$-scores.


In [322]:
par1 = process_paragraph(mod_section_paragraphs[5])
print(par1)

In the second step of the study, genetic data from GWAS was processed. This involved aligning the data and estimating $z$-scores, which are essential for the TWAS process to ensure compatibility with SNPs in predictive models. The necessary scripts for these steps can be found at the following link: https://github.com/hakyimlab/summary-gwas-imputation. These procedures were carried out for 1,000 different traits, resulting in a total of 8,325,729 variants that included both original and estimated $z$-scores.


In [323]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [324]:
display(paragraph_matches[-1])

('supplementary material',
 '**Step 2 - GWAS processing**. These steps include harmonization of GWAS and imputation of $z$-scores, which are part of the TWAS pipeline and are needed in order to ensure an acceptable overlap with SNPs in prediction models. The scripts to run these steps are available in [@url:https://github.com/hakyimlab/summary-gwas-imputation]. These procedures were run for all 1,000 random phenotypes and generated a total number of 8,325,729 variants, including those with original and imputed $z$-scores.',
 'In the second step of the study, genetic data from GWAS was processed. This involved aligning the data and estimating $z$-scores, which are essential for the TWAS process to ensure compatibility with SNPs in predictive models. The necessary scripts for these steps can be found at the following link: https://github.com/hakyimlab/summary-gwas-imputation. These procedures were carried out for 1,000 different traits, resulting in a total of 8,325,729 variants that inc

####  Paragraph 03

In [325]:
par0 = process_paragraph(orig_section_paragraphs[7])
print(par0)

Finally, we ran our GLS model (Equation (@eq:reg:model)) to compute an association between each of the 987 LVs in MultiPLIER and the 1,000 S-MultiXcan results on random phenotypes. For this, we built a gene correlation matrix specifically for this cohort (see [Methods](#sec:methods:reg)). Then, we compared the GLS results with an equivalent, baseline ordinarly least squares (OLS) model assuming independence between genes. Figure @fig:reg:nulls:qqplots compares the distribution of $p$-values of the OLS and GLS models. The GLS model has a slightly smaller mean type I error rate (0.0558, SD=0.0127) than the baseline OLS model (0.0584, SD=0.0140), and $p$-values follow more closely the expected uniform distribution. Importantly, the GLS model is able to correct for LVs with adjacent and highly correlated genes at the top such as LV234 (Figure @fig:reg:nulls:qqplot:lv234), LV847 (Figure @fig:reg:nulls:qqplot:lv847), LV45 (Figure @fig:reg:nulls:qqplot:lv45), or LV800 (Figure @fig:reg:nulls:q

In [326]:
par1 = process_paragraph(mod_section_paragraphs[7])
print(par1)

We used a statistical model to analyze the association between 987 gene expression patterns in MultiPLIER and 1,000 results from S-MultiXcan on random traits. To do this, we created a gene correlation matrix specifically for this group. We then compared the results from our model with a standard model that assumes genes are independent. Figure shows the comparison of $p$-values between the two models. The GLS model had a slightly lower mean type I error rate (0.0558, SD=0.0127) compared to the baseline OLS model (0.0584, SD=0.0140), and the $p$-values were closer to the expected distribution. The GLS model effectively corrected for gene patterns with high correlations, such as LV234, LV847, LV45, and LV800. In contrast, the OLS model had higher errors and lower $p$-values in these cases.


In [327]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [328]:
display(paragraph_matches[-1])

('supplementary material',
 'Finally, we ran our GLS model (Equation (@eq:reg:model)) to compute an association between each of the 987 LVs in MultiPLIER and the 1,000 S-MultiXcan results on random phenotypes. For this, we built a gene correlation matrix specifically for this cohort (see [Methods](#sec:methods:reg)). Then, we compared the GLS results with an equivalent, baseline ordinarly least squares (OLS) model assuming independence between genes. Figure @fig:reg:nulls:qqplots compares the distribution of $p$-values of the OLS and GLS models. The GLS model has a slightly smaller mean type I error rate (0.0558, SD=0.0127) than the baseline OLS model (0.0584, SD=0.0140), and $p$-values follow more closely the expected uniform distribution. Importantly, the GLS model is able to correct for LVs with adjacent and highly correlated genes at the top such as LV234 (Figure @fig:reg:nulls:qqplot:lv234), LV847 (Figure @fig:reg:nulls:qqplot:lv847), LV45 (Figure @fig:reg:nulls:qqplot:lv45), or L

####  Paragraph 04

In [329]:
par0 = process_paragraph(orig_section_paragraphs[8])
print(par0)

We also detected other LVs with higher-than-expected mean type I errors for both the GLS and OLS models, although they don't have a relatively large number of adjacent genes at the top. One example is LV914, shown in Figure @fig:reg:nulls:qqplot:lv914. Inflation in these LVs might be explained by inaccuracies in correlation estimates between the individual-level MultiXcan model and its summary-based version (see Methods). Therefore, we flagged those with a type I error rate larger than 0.07 (127 LVs) and excluded them from our main analyses. We didn't see signs of inflation when applying the method in real data (Figure @fig:reg:real:qqplots).


In [330]:
par1 = process_paragraph(mod_section_paragraphs[8])
print(par1)

We identified additional LVs that had higher-than-expected mean errors in both the GLS and OLS models, even though they did not have a large number of nearby genes at the top. An example of this is LV914, which is depicted in Figure 1. The inflation in these LVs may be due to inaccuracies in correlation estimates between the individual-level MultiXcan model and its summary-based version. As a result, we identified 127 LVs with a type I error rate greater than 0.07 and excluded them from our primary analyses. We did not observe any signs of inflation when applying the method to real data, as shown in Figure 2.


In [331]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [332]:
display(paragraph_matches[-1])

('supplementary material',
 "We also detected other LVs with higher-than-expected mean type I errors for both the GLS and OLS models, although they don't have a relatively large number of adjacent genes at the top. One example is LV914, shown in Figure @fig:reg:nulls:qqplot:lv914. Inflation in these LVs might be explained by inaccuracies in correlation estimates between the individual-level MultiXcan model and its summary-based version (see Methods). Therefore, we flagged those with a type I error rate larger than 0.07 (127 LVs) and excluded them from our main analyses. We didn't see signs of inflation when applying the method in real data (Figure @fig:reg:real:qqplots).",
 'We identified additional LVs that had higher-than-expected mean errors in both the GLS and OLS models, even though they did not have a large number of nearby genes at the top. An example of this is LV914, which is depicted in Figure 1. The inflation in these LVs may be due to inaccuracies in correlation estimates b

####  Paragraph 05

In [333]:
par0 = process_paragraph(orig_section_paragraphs[33])
print(par0)

For the first scenario, we shuffled genes in $\mathbf{M}$ for each trait, and this randomized matrix was then projected into the latent space. For the second scenario, we projected matrix $\mathbf{M}$ into the latent space, and then shuffled LVs in $\hat{\mathbf{M}}$ for each trait. For each of these scenarios, we ran exactly the same clustering pipeline we used for the real data ([Methods](#sec:methods:clustering)), generating an ensemble of partitions that was later combined using the same consensus functions to derive the final partitions of traits. Finally, we computed 1) stability statistics on the ensemble partitions from different algorithms and 2) the agreement of the final consensus partition with the ensemble.


In [334]:
par1 = process_paragraph(mod_section_paragraphs[33])
print(par1)

In the first scenario, we rearranged genes in matrix M for each trait, then projected this rearranged matrix into the hidden space. In the second scenario, we projected matrix M into the hidden space, and then rearranged latent variables in M-hat for each trait. In both scenarios, we used the same clustering process as we did for the actual data, creating multiple partitions that were later combined using consensus functions to determine the final trait groupings. We then calculated stability statistics on the different algorithm partitions and measured the agreement of the final consensus partition with the ensemble.


In [335]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [336]:
display(paragraph_matches[-1])

('supplementary material',
 'For the first scenario, we shuffled genes in $\\mathbf{M}$ for each trait, and this randomized matrix was then projected into the latent space. For the second scenario, we projected matrix $\\mathbf{M}$ into the latent space, and then shuffled LVs in $\\hat{\\mathbf{M}}$ for each trait. For each of these scenarios, we ran exactly the same clustering pipeline we used for the real data ([Methods](#sec:methods:clustering)), generating an ensemble of partitions that was later combined using the same consensus functions to derive the final partitions of traits. Finally, we computed 1) stability statistics on the ensemble partitions from different algorithms and 2) the agreement of the final consensus partition with the ensemble.',
 'In the first scenario, we rearranged genes in matrix M for each trait, then projected this rearranged matrix into the hidden space. In the second scenario, we projected matrix M into the hidden space, and then rearranged latent varia

####  Paragraph 06

In [337]:
par0 = process_paragraph(orig_section_paragraphs[35])
print(par0)

The results of this analysis (Figure @fig:sup:clustering:agreement) show that, under the two simulated null scenarios, the agreement of the consensus partitions with the ensemble is very close to zero. This means, as expected, that there is no consensus among ensemble partitions generated with different clustering algorithms and data representations. In contrast, using the real data, the consensus clustering approach finds trait pairs that are grouped together across the different members of the ensemble. The partitions above the 75th percentile were considered in the main analyses, and are shown in the clustering tree in Figure @fig:clustering:tree.


In [338]:
par1 = process_paragraph(mod_section_paragraphs[35])
print(par1)

The findings from this analysis (Figure 1) indicate that, in the simulated null scenarios, there is minimal agreement between the consensus partitions and the ensemble. This suggests that different clustering algorithms and data representations do not produce consistent results. However, when using actual data, the consensus clustering method identifies trait pairs that consistently group together across different ensemble members. For the main analyses, only partitions above the 75th percentile were considered, and these are displayed in the clustering tree in Figure 2.


In [339]:
paragraph_matches.append(
    (
        section_name,
        par0,
        par1,
    )
)

In [340]:
display(paragraph_matches[-1])

('supplementary material',
 'The results of this analysis (Figure @fig:sup:clustering:agreement) show that, under the two simulated null scenarios, the agreement of the consensus partitions with the ensemble is very close to zero. This means, as expected, that there is no consensus among ensemble partitions generated with different clustering algorithms and data representations. In contrast, using the real data, the consensus clustering approach finds trait pairs that are grouped together across the different members of the ensemble. The partitions above the 75th percentile were considered in the main analyses, and are shown in the clustering tree in Figure @fig:clustering:tree.',
 'The findings from this analysis (Figure 1) indicate that, in the simulated null scenarios, there is minimal agreement between the consensus partitions and the ensemble. This suggests that different clustering algorithms and data representations do not produce consistent results. However, when using actual d

# Close connections

In [341]:
g.close()

# Save

In [342]:
len(paragraph_matches)

63

In [343]:
paragraph_matches[:2]

[('abstract',
  'Genes act in concert with each other in specific contexts to perform their functions. Determining how these genes influence complex traits requires a mechanistic understanding of expression regulation across different conditions. It has been shown that this insight is critical for developing new therapies. Transcriptome-wide association studies have helped uncover the role of individual genes in disease-relevant mechanisms. However, modern models of the architecture of complex traits predict that gene-gene interactions play a crucial role in disease origin and progression. Here we introduce PhenoPLIER, a computational approach that maps gene-trait associations and pharmacological perturbation data into a common latent representation for a joint analysis. This representation is based on modules of genes with similar expression patterns across the same conditions. We observe that diseases are significantly associated with gene modules expressed in relevant cell types, an

In [344]:
df = pd.DataFrame(paragraph_matches).rename(
    columns={
        0: "section",
        1: "original",
        2: "modified",
    }
)

In [345]:
df.shape

(63, 3)

In [346]:
df.head()

,section,original,modified
0,abstract,Genes act in concert with each other in specif...,How do genes interact to influence complex tra...
1,introduction,Genes work together in context-specific networ...,Genes work together in specific networks to ca...
2,introduction,Given the availability of gene expression data...,With the abundance of gene expression data ava...
3,introduction,These gene-gene interactions play a crucial ro...,Gene-gene interactions are essential in curren...
4,introduction,"Here we propose PhenoPLIER, an omnigenic appro...",In this paper titled 'Projecting genetic assoc...


In [347]:
df.to_pickle(OUTPUT_FILE_PATH)

# Reverse original/modified columns

In [348]:
df_reversed = df.rename(columns={"original": "modified2"}).rename(
    columns={"modified": "original", "modified2": "modified"}
)

In [349]:
df_reversed.shape

(63, 3)

In [350]:
df_reversed.head()

,section,modified,original
0,abstract,Genes act in concert with each other in specif...,How do genes interact to influence complex tra...
1,introduction,Genes work together in context-specific networ...,Genes work together in specific networks to ca...
2,introduction,Given the availability of gene expression data...,With the abundance of gene expression data ava...
3,introduction,These gene-gene interactions play a crucial ro...,Gene-gene interactions are essential in curren...
4,introduction,"Here we propose PhenoPLIER, an omnigenic appro...",In this paper titled 'Projecting genetic assoc...


## Save

In [351]:
df_reversed.to_pickle(REVERSED_OUTPUT_FILE_PATH)